## **Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import os
import re
import sqlite3
from pandas.tseries.offsets import MonthEnd
pd.set_option('display.max_columns',50)

date = 202112   # Periode data

## **Read LSMK File**

In [2]:
#Produk BI - Form 4
coll = ['FKBIBranch', 'Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - mi13']
fields = ['Branch', 'Valuta', 'JatuhTempo', 'Os']
Form4 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS6_1.xlsx', usecols=coll)
Form4.columns = fields
Form4['Form'] = 4
Form4['KatPort'] = 10
Form4['Kol'] = 1
Form4['Produk'] = 'BI'

# Produk Bank - Form 5
coll = ['FKBIBranch','Jenis Instrumen - si18','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10',
        'Peringkat - Nilai Peringkat - si19','Jumlah - Bulan Laporan - mi25',
        'Kualitas - si6017','CKPN - Individual - mi32','CKPN - Kolektif - mi33']
fields = ['Branch', 'Instrumen', 'Valuta', 'JatuhTempo','NilaiPeringkat', 'Os', 'Kol', 
          'CkpnIndividu', 'CkpnKol']
Form5 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS48_1.xlsx', usecols=coll)
Form5.columns = fields
Form5['Form'] = 5
Form5['KatPort'] = np.where(Form5['Instrumen']==10,'14','15')
Form5['Produk'] = 'BANK'

#Produk Surat Berharga
coll = ['FKBIBranch','Nomor Nasabah - si68','Jenis Instrumen - si42','Jenis Valuta - si2','Jangka waktu - Jatuh Tempo - di10',
        'Jumlah - Bulan Laporan - mi55','Imbalan yang akan diterima - mi57','Metode Pengukuran - si44', 'Kategori Portofolio - si37',
        'Peringkat - Nilai Peringkat - si19','Kualitas - si6017','CKPN - Individual - mi61', 'CKPN - Kolektif - mi62']
fields = ['Branch','NoNas','Instrumen','Valuta', 'JatuhTempo','MetodeUkur','KatPort','NilaiPeringkat','Os','Imbalan',
          'Kol','CkpnIndividu','CkpnKol']

Form7 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS52_1.xlsx', usecols=coll)
Form7.columns = fields
Form7 = Form7.replace(to_replace='1/1/1900', value='')

filt = ~(Form7['KatPort'] == 36)
Form7 = Form7.loc[filt]

filt = (Form7['MetodeUkur'] == 3)
Form7 = Form7.loc[filt]

Form7 = Form7.drop_duplicates(subset=['NoNas', 'Os'], keep='first')
Form7.reset_index(drop=True,inplace=True)
Form7['Form'] = 7
Form7['Produk'] = 'SURAT BERHARGA'

#Produk Akseptasi
coll = ['FKBIBranch','Jenis valuta - si2', 'Jangka Waktu - Jatuh Tempo - di10','Kategori Portofolio - si37',
        'Peringkat - Nilai Pemeringkat - si19','Jumlah - Bulan Laporan - mi78','Kualitas - si58', 
        'Cadangan Kerugian Penurunan Nilai - Individual - mi81','Cadangan Kerugian Penurunan Nilai - Kolektif - mi82']
fields = ['Branch','Valuta','JatuhTempo','KatPort','NilaiPeringkat','Os','Kol','CkpnIndividu','CkpnKol']
Form9 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS54_1.xlsx', usecols=coll)
Form9.columns = fields
Form9['Form'] = 9
Form9['Produk'] = 'AKSEPTASI'

#Produk Murabahah
coll = ['FKBIBranch', 'Nomor Rekening - si188','Jenis Valuta - si2','Jenis Piutang - si85',
        'Jangka Waktu - Jatuh Tempo - di10','Sektor Ekonomi - si394',
        'Kategori Portofolio - si140', 'Peringkat - Nilai Peringkat - si19',
        'Saldo Harga Pokok - mi89','Imbalan yang Akan Diterima - mi96',
        'Kualitas - si58', 'CKPN - Individual - mi97','CKPN - Kolektif - mi98']
fields = ['Branch','NoRek', 'Valuta', 'JatuhTempo','Instrumen', 'SekEko', 'KatPort', 
          'NilaiPeringkat','Os','Imbalan','Kol', 'CkpnIndividu','CkpnKol']
Form10 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS56_1.xlsx', usecols=coll)
Form10.columns = fields
Form10['Imbalan'] = np.where((Form10['KatPort']==60) |
                             (Form10['KatPort']==62), 0, 
                                   Form10['Imbalan'])
Form10 = Form10.drop_duplicates(subset=['NoRek', 'Os'], keep='first')
Form10['Form'] = 10
Form10['Produk'] = 'MURABAHAH'

#Produk Istishna
coll = ['FKBIBranch','Nomor Rekening - si188','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10',
                 'Jenis Piutang - si99','Sektor Ekonomi - si394','Kategori Portofolio - si140',
                 'Peringkat - Nilai Peringkat - si19','Saldo Harga Pokok - mi101',
                 'Imbalan yang Akan Diterima - mi108','Kualitas - si58',
                  'CKPN - Individual - mi109','CKPN - Kolektif - mi110']
fields = ['Branch','NoRek','Valuta','JatuhTempo','Instrumen','SekEko', 'KatPort','NilaiPeringkat', 'Os', 'Imbalan','Kol','CkpnIndividu', 'CkpnKol']
Form11 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS58_1.xlsx', usecols=coll)
Form11.columns = fields
Form11['Imbalan'] = np.where((Form11['KatPort']==60) |
                             (Form11['KatPort']==62), 0, 
                                   Form11['Imbalan'])
Form11 = Form11.drop_duplicates(subset=['NoRek', 'Os'], keep='first')
Form11['Form'] = 11
Form11['Produk'] = 'ISTISHNA'

#Produk Qardh 
coll = ['FKBIBranch','Nomor Rekening - si188','Jenis Valuta - si2', 'Jangka Waktu - Jatuh Tempo - di10',
        'Jenis Piutang - si111','Sektor Ekonomi - si394','Kategori Portofolio - si140',
        'Peringkat - Nilai Peringkat - si19','Jumlah - Bulan Laporan - mi114','Imbalan yang AKan Diterima - mi118',
        'Kualitas - si58', 'CKPN - Individual - mi119','CKPN - Kolektif - mi120']
fields = ['Branch','NoRek','Valuta','JatuhTempo','Instrumen','SekEko', 'KatPort', 'NilaiPeringkat', 'Os', 
          'Imbalan','Kol','CkpnIndividu', 'CkpnKol']
Form12 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS62_1.xlsx', usecols=coll)
Form12.columns = fields
Form12['Imbalan'] = np.where((Form12['KatPort']==60) |
                             (Form12['KatPort']==62), 0, 
                                   Form12['Imbalan'])
Form12 = Form12.drop_duplicates(subset=['NoRek', 'Os'], keep='first')
Form12['Form'] = 12
Form12['Produk'] = 'QARDH'

#produk Mudharabah_Musyarakah
coll = ['FKBIBranch','Nomor Rekening - si188','Jenis Valuta - si2', 'Jangka Waktu - Jatuh Tempo - di10','Jenis Pembiayaan - si121',
        'Sektor Penggunaan - si394','Kategori Portofolio - si140',
        'Peringkat - Nilai Peringkat - si19','Jumlah - Bulan Laporan - mi128',
        'Imbalan yang Akan Diterima - mi135','Kualitas - si58',
        'CKPN - Individual - mi136','CKPN - Kolektif - mi137']
fields = ['Branch','NoRek','Valuta','JatuhTempo','Instrumen','SekEko', 'KatPort','NilaiPeringkat', 'Os', 'Imbalan','Kol','CkpnIndividu', 'CkpnKol']
Form13 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS60_1.xlsx', usecols=coll)
Form13.columns = fields
Form13['Imbalan'] = np.where((Form13['KatPort']==60) |
                             (Form13['KatPort']==62), 0, 
                                   Form13['Imbalan'])
Form13 = Form13.drop_duplicates(subset=['NoRek', 'Os'], keep='first')
Form13['Form'] = 13
Form13['Produk'] = 'MUDHARABAH_MUSYARAKAH'

#Produk Ijarah
coll = ['FKBIBranch','Nomor Rekening - si188','Jenis Valuta - si2','Jangka waktu - Jatuh tempo - di10', 
        'Jenis Pembiayaan - si138','Sektor Ekonomi - si394','Kategori Portofolio - si140','Peringkat - Nilai Peringkat - si19',
        'Kategori Akad - Perolehan Aset - Harga Perolehan - mi144', 'Imbalan yang akan diterima - mi155','Kualitas - si58', 
        'CKPN - Individual - mi156','CKPN - Kolektif - mi157','Akumulasi Penyusutan / Amortisasi - mi150']
fields = ['Branch', 'NoRek', 'Valuta', 'JatuhTempo', 'Instrumen', 'SekEko', 'KatPort', 'NilaiPeringkat', 'Os','Amortisasi', 
          'Imbalan','Kol','CkpnIndividu', 'CkpnKol']
Form14 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS64_1.xlsx', usecols=coll)
Form14.columns = fields
Form14['Os'] = Form14['Os'] - Form14['Amortisasi']
Form14['Imbalan'] = np.where((Form14['KatPort']==60) |
                             (Form14['KatPort']==62), 0, 
                                   Form14['Imbalan'])
Form14 = Form14.drop_duplicates(subset=['NoRek', 'Os'], keep='first')
Form14['Form'] = 14
Form14['Produk'] = 'IJARAH'

#Produk Kewajiban pada BI
coll = ['FKBIBranch','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - mi258']
fields = ['Branch','Valuta', 'JatuhTempo','Os']
Form31 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS23_1.xlsx', usecols=coll)
Form31.columns = fields
Form31['Form'] = 31
Form31['Produk'] = '09.DUE TO BI'

#Produk Kewajiban pada Bank Lain
coll = ['FKBIBranch','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - Bulan Laporan - mi264']
fields = ['Branch','Valuta','JatuhTempo','Os']
Form32 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS24_1.xlsx', usecols=coll)
Form32.columns = fields
Form32['Form'] = 32
Form32['Produk'] = '10.DUE TO OTHERS'

#Produk Kewajiban pada Bank Lain
coll = ['FKBIBranch','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - Bulan Laporan - mi275']
fields = ['Branch','Valuta','JatuhTempo','Os']
Form34 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS26_1.xlsx', usecols=coll)
Form34.columns = fields
Form34['Form'] = 34
Form34['Produk'] = '11.SECURITY ISSUED'

#Produk Pembiayaan Di Terima
coll = ['FKBIBranch','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - Bulan Laporan - mi291']
fields = ['Branch','Valuta','JatuhTempo','Os']
Form36 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS28_1.xlsx', usecols=coll)
Form36.columns = fields
Form36['Form'] = 36
Form36['Produk'] = '12.BORROWING'

#Produk Pembiayaan Di Terima
coll = ['FKBIBranch','Jenis Valuta - si2','Jangka Waktu - Jatuh Tempo - di10','Jumlah - mi379']
fields = ['Branch','Valuta','JatuhTempo','Os']
Form45 = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMS35_1.xlsx', usecols=coll)
Form45.columns = fields
Form45['Form'] = 45
Form45['Produk'] = '13.IRRECOVABLE'
Form45.head()

#Produk Penyertaan
Bsmg = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMG1_1.xlsx')
Bsmg=Bsmg[['FKBIBranch', 'BIText', 'SandiBI','Jumlah']]

sandibi=[200]
Form15 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form15 = Form15.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
list_drop = ['BIText','SandiBI']
Form15.drop(list_drop, axis=1, inplace=True)
Form15['Form'] = 15
Form15['Produk'] = 'PENYERTAAN'

#Produk Terbengkalai
sandibi=[217]
Form21 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form21 = Form21.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form21.drop(drop_list, axis=1, inplace=True)
Form21['Form'] = 21
Form21['Produk'] = 'PROPERTI_TERBENGKALAI'

#Produk AYDA
sandibi=[218]
Form22 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form22 = Form22.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form22.drop(drop_list, axis=1, inplace=True)
Form22['Form'] = 22
Form22['Produk'] = 'ASET_YG_DI_AMBIL ALIH'

#Produk Rekening Tunda
sandibi=[219]
Form23 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form23 = Form23.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form23.drop(drop_list, axis=1, inplace=True)
Form23['Form'] = 23
Form23['Produk'] = 'REKENING_TUNDA'

#Produk Kas
sandibi=[100]
Form3 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form3 = Form3.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form3.drop(drop_list, axis=1, inplace=True)
Form3['Form'] = 3
Form3['Produk'] = 'KAS'

#Produk Aset tetap dan Inventaris Neto
sandibi=[215,216]
Form20 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form20 = Form20.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
Form20['Os'] = Form20['Os']-Form20['Os'].shift(1)
Form20.dropna(axis=0, how='any', inplace=True)
drop_list = ['BIText','SandiBI']
Form20.drop(drop_list, axis=1, inplace=True)
Form20['Form'] = 20
Form20['Produk'] = 'ASET_TETAP_DAN INVENTARIS NETO'

#Produk Spot Forward
sandibi=[135]
Form6 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form6 = Form6.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form6.drop(drop_list, axis=1, inplace=True)
Form6['Form'] = 6
Form6['Produk'] = 'TAGIHAN SPOT FORWARD'

#Produk Rupa-rupa
sandibi=[230]
Form28 = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form28 = Form28.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Os')
drop_list = ['BIText','SandiBI']
Form28.drop(drop_list, axis=1, inplace=True)
Form28['Form'] = 28
Form28['Produk'] = 'RUPA_RUPA'

#Produk CKPN BSMG
sandibi=[225]
Ckpn_bsmg = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Ckpn_bsmg = Ckpn_bsmg.groupby(['BIText','SandiBI'])['Jumlah'].sum().reset_index(name='Ckpn_Bsmg')
drop_list = ['BIText','SandiBI']
Ckpn_bsmg.drop(drop_list, axis=1, inplace=True)
Ckpn_bsmg['Produk'] = 'Lain_lain'

#Lain-lain
lain = pd.concat([Form6,Form21,Form23,Form28])
lain['Produk'] = 'Lain_lain'
lain = lain.groupby(['Produk'])['Os'].sum().reset_index(name='lain')

#RSL NONRSL
Bsmg = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/LSMK/BSMG1_1.xlsx')
Bsmg=Bsmg[['FKBIBranch', 'BIText', 'SandiBI','ValueR','ValueV']]

sandibi=[320,301]
Form99r = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form99r = Form99r.groupby(['BIText'])['ValueR'].sum().reset_index(name='Os')
list_drop = ['BIText']
Form99r.drop(list_drop, axis=1, inplace=True)
Form99r['Valuta'] = 'IDR'

Form99v = Bsmg[Bsmg['SandiBI'].isin(sandibi)]
Form99v = Form99v.groupby(['BIText'])['ValueV'].sum().reset_index(name='Os')
list_drop = ['BIText']
Form99v.drop(list_drop, axis=1, inplace=True)
Form99v['Valuta'] = 'Val'
Form99 = pd.concat([Form99r,Form99v])
Form99['Form'] = 99
Form99['Produk'] = '06.CURRENT_ACCOUNT'


# Form7.head()

## **create list publik exposure**

In [3]:
list_pubex = pd.concat([Form3,Form4,Form5,Form7,Form9,Form10,Form11,Form12,Form13,Form14,Form15,Form20,Form22])

list_pubex['JatuhTempo'] = list_pubex['JatuhTempo'].replace(to_replace='1/1/1900', value='')
list_pubex['JatuhTempo']=pd.to_datetime(list_pubex['JatuhTempo'], format='%Y-%m-%d')
list_pubex['Branch'] = list_pubex['Branch'].fillna(147057000)
list_pubex['Valuta'] = list_pubex['Valuta'].fillna("IDR")
list_pubex['KatPort'] = list_pubex['KatPort'].fillna(99)
list_pubex['SekEko'] = list_pubex['SekEko'].fillna(9999999)
list_pubex['JatuhTempo'] = list_pubex['JatuhTempo'].fillna('1900-01-01')
list_pubex['Kol'] = list_pubex['Kol'].fillna(1)

list_pubex['Date'] = date
list_pubex['EndOfMonth'] = pd.to_datetime(list_pubex['Date'], format="%Y%m") + MonthEnd(1)

def myfillna(series):
    if series.dtype is np.dtype(float):
        return series.fillna(0)
    elif series.dtype is np.dtype(object):
        return series.fillna('')
    else:
        return series
list_pubex = list_pubex.apply(myfillna)

#imbalan
Imbalan = list_pubex.groupby(['Date'])['Imbalan'].sum().reset_index(name='Imbalan')
Imbalan['Produk'] = 'Lain_lain'
# Katport No.11 Lain
Lain2 = Ckpn_bsmg.merge(Imbalan, on='Produk', how='left').merge(lain, on='Produk', how='left')
Lain2['Os'] = Lain2['lain']-Lain2['Ckpn_Bsmg']-Lain2['Imbalan']
Lain2['Produk'] = 'Lain_Lain'
Lain2=Lain2[['Date', 'Produk', 'Os']]

#gabungan list publikasi exposure
list_pubexp = pd.concat([list_pubex, Lain2])

list_pubexp[['JatuhTempo']] = list_pubexp[['JatuhTempo']].apply(pd.to_datetime)
list_pubexp['Sisa']=list_pubexp['JatuhTempo']-list_pubexp['EndOfMonth']
list_pubexp['Sisa']=list_pubexp['Sisa']/np.timedelta64(1,'D')
list_pubexp['Sisa'].fillna(value=0, inplace=True)
list_pubexp['Ckpn'] = list_pubexp['CkpnIndividu']+list_pubexp['CkpnKol']
list_pubexp['Tagihan'] = list_pubexp['Os']+list_pubexp['Imbalan']
list_pubexp['TagihanBersih'] = np.where(list_pubexp['Kol']==1, list_pubexp['Tagihan'], 
                                   list_pubexp['Tagihan']-list_pubexp['Ckpn'])

def func(row):
    if row['Sisa']<= 0:
        val = '05.NonKontraktual'
    elif (row['Sisa'] > 0 and row['Sisa'] <= 360):
        val = '01.<=1Tahun'
    elif (row['Sisa'] > 360 and row['Sisa'] <= 1080):
        val = '02.> 1Tahun S.D. 3Tahun'
    elif (row['Sisa'] > 1080 and row['Sisa'] <= 1800):
        val = '03.> 3Tahun S.D. 5Tahun'
    else:
        val = '04.> 5Tahun'
    return val
list_pubexp['KelSisa'] = list_pubexp.apply(func, axis=1)
list_pubexp['KelSisa'].fillna(value='05.NonKontraktual', inplace=True)
list_pubexp = list_pubexp.sort_values(['TagihanBersih'], ascending=[0]) 

#Katport
Katport = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/katport1.xlsx',index_col=0)
#Sekeko
Sekeko = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/sekeko1.xlsx',index_col=0)
#Branch
Branch = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/branch1.xlsx',index_col=0)
#Rating
Rating = pd.read_excel('C:/Users/20150503/Downloads/PubExp_py/rating1.xlsx',index_col=0)

list_pubexp = pd.merge(list_pubexp,Katport,left_on="KatPort",right_on="KatPort",how="left")
list_pubexp = pd.merge(list_pubexp,Sekeko,left_on="SekEko",right_on="SekEko",how="left")
list_pubexp = pd.merge(list_pubexp,Branch,left_on="Branch",right_on="Branch",how="left")
list_pubexp = pd.merge(list_pubexp,Rating,left_on="NilaiPeringkat",right_on="Nilai",how="left")
# inplace=True makes the change to the data
list_pubexp['Ket_Katport'].fillna(value='11. Aset Lainnya/ Others Asset', inplace=True)
list_pubexp['Ket_Katport_final'].fillna(value='11. Aset Lainnya/ Others Asset', inplace=True)
list_pubexp['Ket_Sekeko'].fillna(value='20. Lainnya/ Others', inplace=True)
list_pubexp['Kel_Rating'].fillna(value='8.Tanpa Peringkat', inplace=True)
list_pubexp['Area'].fillna(value='05.Lainnya', inplace=True)
list_pubexp['Kol'].fillna(value=1, inplace=True)

list_pubexp.head()

,Os,Form,Produk,Branch,Valuta,JatuhTempo,KatPort,Kol,Instrumen,NilaiPeringkat,CkpnIndividu,CkpnKol,NoNas,MetodeUkur,Imbalan,NoRek,SekEko,Amortisasi,Date,EndOfMonth,Sisa,Ckpn,Tagihan,TagihanBersih,KelSisa,Ket_Katport,Ket_Katport_final,Ket_Sekeko,Area,Kel_Rating
0,1.305107e+13,7.0,SURAT BERHARGA,147001000.0,IDR,2041-11-15,16,1.0,80.0,3.0,0.0,0.000000e+00,5443909.0,3.0,0.0,,9999999.0,0.0,202112,2021-12-31,7259.0,0.000000e+00,1.305107e+13,1.305107e+13,04.> 5Tahun,02. Tagihan Kepada Entitas Sektor Publik/ Cl...,02. Tagihan Kepada Entitas Sektor Publik/ Cl...,20. Lainnya/ Others,01.Jabodetabek,3.idA+ s.d id A-
1,8.000000e+12,7.0,SURAT BERHARGA,147001000.0,IDR,2038-06-29,35,1.0,80.0,0.0,0.0,9.975000e+09,999000111.0,3.0,0.0,,9999999.0,0.0,202112,2021-12-31,6024.0,9.975000e+09,8.000000e+12,8.000000e+12,04.> 5Tahun,09. Tagihan kepada Korporasi/ Claim to Corporate,09. Tagihan kepada Korporasi/ Claim to Corporate,20. Lainnya/ Others,01.Jabodetabek,8.Tanpa Peringkat
2,3.605444e+12,4.0,BI,147001000.0,IDR,1900-01-01,10,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,,9999999.0,0.0,202112,2021-12-31,-44559.0,0.000000e+00,3.605444e+12,3.605444e+12,05.NonKontraktual,01. Tagihan Kepada Pemerintah/ Claim to The G...,01. Tagihan Kepada Pemerintah/ Claim to The G...,20. Lainnya/ Others,01.Jabodetabek,8.Tanpa Peringkat
3,2.843781e+12,20.0,ASET_TETAP_DAN INVENTARIS NETO,147057000.0,IDR,1900-01-01,99,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,,9999999.0,0.0,202112,2021-12-31,-44559.0,0.000000e+00,2.843781e+12,2.843781e+12,05.NonKontraktual,11. Aset Lainnya/ Others Asset,11. Aset Lainnya/ Others Asset,20. Lainnya/ Others,01.Jabodetabek,8.Tanpa Peringkat
4,1.431000e+12,4.0,BI,147001000.0,IDR,2022-01-03,10,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,,9999999.0,0.0,202112,2021-12-31,3.0,0.000000e+00,1.431000e+12,1.431000e+12,01.<=1Tahun,01. Tagihan Kepada Pemerintah/ Claim to The G...,01. Tagihan Kepada Pemerintah/ Claim to The G...,20. Lainnya/ Others,01.Jabodetabek,8.Tanpa Peringkat


## **Create List BRBB**

In [4]:
list_brbb = pd.concat([Form4,Form5,Form7,Form10,Form11,Form12,Form13,Form14,Form99,Form31,Form32,Form34,Form36,Form45])
list_brbb['Date'] = date
list_brbb['EndOfMonth'] = pd.to_datetime(list_brbb['Date'], format="%Y%m") + MonthEnd(1)
list_brbb['JatuhTempo'].replace(['1/1/1900',0],'1900-1-1', inplace=True)
list_brbb[['JatuhTempo']] = list_brbb[['JatuhTempo']].apply(pd.to_datetime)
list_brbb['Sisa']=list_brbb['JatuhTempo']-list_brbb['EndOfMonth']
list_brbb['Sisa']=list_brbb['Sisa']/np.timedelta64(1,'D')
list_brbb['Sisa'].fillna(value=0, inplace=True)

list_brbb['Valuta'] = ['Rupiah' if x =='IDR' else 'Valas' for x in list_brbb['Valuta']]

list_brbb['Amortisasi'].fillna(value=0, inplace=True)
list_brbb['Os'].fillna(value=0, inplace=True)
list_brbb['Imbalan'].fillna(value=0, inplace=True)
list_brbb['CkpnIndividu'].fillna(value=0, inplace=True)
list_brbb['CkpnKol'].fillna(value=0, inplace=True)

list_brbb['Ckpn'] = list_brbb['CkpnIndividu']+list_brbb['CkpnKol']
list_brbb['Tagihan'] = list_brbb['Os']+list_brbb['Imbalan']
list_brbb['TagihanBersih'] = np.where(list_brbb['Kol']==1, list_brbb['Tagihan'], 
                                   list_brbb['Tagihan']-list_brbb['Ckpn'])
list_brbb = list_brbb.apply(myfillna)


In [5]:
def func1(row):
    if row['Sisa']<= 0:
        val = '14.NON_RSA'
    elif (row['Sisa'] > 0 and row['Sisa'] <= 30):
        val = '01.0-1 MTH'
    elif (row['Sisa'] > 30 and row['Sisa'] <= 90):
        val = '02.1-3 MTH'
    elif (row['Sisa'] > 90 and row['Sisa'] <= 180):
        val = '03.3-6 MTH'
    elif (row['Sisa'] > 180 and row['Sisa'] <= 360):
        val = '04.6-12 MTH'
    elif (row['Sisa'] > 360 and row['Sisa'] <= 720):
        val = '05.1-2 YR'
    elif (row['Sisa'] > 720 and row['Sisa'] <= 1080):
        val = '06.2-3 YR'
    elif (row['Sisa'] > 1080 and row['Sisa'] <= 1440):
        val = '07.3-4 YR'
    elif (row['Sisa'] > 1440 and row['Sisa'] <= 1800):
        val = '08.4-5 YR'
    elif (row['Sisa'] > 1800 and row['Sisa'] <= 2520):
        val = '09.5-7 YR'
    elif (row['Sisa'] > 2520 and row['Sisa'] <= 3600):
        val = '10.7-10 YR'
    elif (row['Sisa'] > 3600 and row['Sisa'] <= 5400):
        val = '11.10-15 YR'
    elif (row['Sisa'] > 5400 and row['Sisa'] <= 7200):
        val = '12.15-20 YR'
    else:
        val = '13.> 20 YR'
    return val
list_brbb['KelSisa'] = list_brbb.apply(func1, axis=1)
list_brbb['KelSisa'].fillna(value='05.NonKontraktual', inplace=True)
list_brbb = list_brbb.sort_values(['TagihanBersih'], ascending=[0])
list_brbb.head()

# list_pubexp.head()
BRBB = list_brbb.loc[:,["Produk","Valuta","Tagihan","KelSisa"]]
BRBB['Produk_Val'] = BRBB[['Valuta','Produk']].apply(lambda x: '-'.join(x), axis = 1)

BRBB= pd.pivot_table(BRBB,
            index=['Produk_Val'],
            columns=['KelSisa'],
            aggfunc=[np.sum],
            values=['Tagihan'],
            fill_value=0,
            margins=True,
            dropna=True)

                                        sum                                \
                                    Tagihan                                 
KelSisa                          01.0-1 MTH     02.1-3 MTH     03.3-6 MTH   
Produk_Val                                                                  
Rupiah-06.CURRENT_ACCOUNT                 0              0              0   
Rupiah-10.DUE TO OTHERS        342825629739   326877432697    86882499996   
Rupiah-11.SECURITY ISSUED                 0              0              0   
Rupiah-12.BORROWING                       0              0   100000000000   
Rupiah-13.IRRECOVABLE            6329071364     8334763631              0   
Rupiah-BANK                               0              0              0   
Rupiah-BI                     1431000000000              0              0   
Rupiah-IJARAH                             0        1473189              0   
Rupiah-ISTISHNA                           0              0              0   

## Print Sheet 1.1 - 1.7

In [16]:
import pandas as pd
import numpy as np
import os
import re

from pandas.tseries.offsets import MonthEnd

pd.set_option('display.max_columns',50)

# list_pubexp.head()
KPMM_sheet_A = list_pubexp[["Ket_Katport","Produk","Tagihan","TagihanBersih","Ckpn","Imbalan"]]
KPMM_sheet_A= pd.pivot_table(KPMM_sheet_A,
            index=['Ket_Katport','Produk'],
#             columns=['Area'],
            aggfunc=[np.sum],
            values=['Tagihan','TagihanBersih','Ckpn','Imbalan'],
            fill_value=0,
            margins=True,
            dropna=True)
KPMM_sheet_A1 = list_pubexp[["Ket_Katport_final","Tagihan","TagihanBersih","Ckpn","Imbalan","Os"]]
KPMM_sheet_A1= pd.pivot_table(KPMM_sheet_A1,
            index=['Ket_Katport_final'],
#             columns=['Area'],
            aggfunc=[np.sum],
            values=['Tagihan','TagihanBersih','Ckpn','Imbalan','Os'],
            fill_value=0,
            margins=True,
            dropna=True)
# print(KPMM_sheet_A)
list_1_1 = list_pubexp[["Ket_Katport_final","Area","TagihanBersih"]]
list_1_1 = pd.pivot_table(list_1_1,
            index=['Ket_Katport_final'],
            columns=['Area'],
            aggfunc=[np.sum],
            values=['TagihanBersih'],
            fill_value=0,
            margins=True,
            dropna=True)
# print(List_1_1)
list_1_2 = list_pubexp[["Ket_Katport_final","KelSisa","TagihanBersih"]]                  
list_1_2=pd.pivot_table(list_1_2, 
            index=['Ket_Katport_final'],
            columns=['KelSisa'],
            aggfunc=[np.sum],
            values=['TagihanBersih'],
            fill_value=0,
            margins=True,
            dropna=True)
# print(List_1_2)
list_1_3 = list_pubexp[["Ket_Katport_final","Ket_Sekeko","TagihanBersih"]]                     
list_1_3=pd.pivot_table(list_1_3, 
            index=['Ket_Sekeko'],
            columns=['Ket_Katport_final'],
            aggfunc=[np.sum],
            values=['TagihanBersih'],
            fill_value=0,
            margins=True,
            dropna=True)
# print(List_1_3)

Tag = list_pubexp.groupby('Area')['Tagihan'].sum()

Tag_kol1 = list_pubexp[list_pubexp.Kol<5].copy()
Tag_kol1 = Tag_kol1.groupby('Area')['TagihanBersih'].sum()

Tag_kol2 = list_pubexp[list_pubexp.Kol==5].copy()
Tag_kol2 = Tag_kol2.groupby('Area')['TagihanBersih'].sum()

CkpnInd = list_pubexp.groupby('Area')['CkpnIndividu'].sum()
CkpnKol = list_pubexp.groupby('Area')['CkpnKol'].sum()

list_1_4 = pd.concat([Tag,Tag_kol1,Tag_kol2,CkpnInd,CkpnKol],axis=1,sort=False).reset_index()
list_1_4.reset_index(drop=True, inplace=True)
list_1_4_cols = ['Area', '01.Tag', '02.Tag1', '03.Tag5', '04.CkpnInd', '05.CkpnKol']
list_1_4.columns = list_1_4_cols
list_1_4=pd.pivot_table(list_1_4, 
            #index=['KetSekEko'],
            columns=['Area'],
            aggfunc=[np.sum],
            values=['01.Tag', '02.Tag1', '03.Tag5', '04.CkpnInd', '05.CkpnKol'],
            fill_value=0,
            #margins=True,
            dropna=True)
# List_1_4
Tag = list_pubexp.groupby('Ket_Sekeko')['Tagihan'].sum()

Tag_kol1 = list_pubexp[list_pubexp.Kol<5].copy()
Tag_kol1 = Tag_kol1.groupby('Ket_Sekeko')['TagihanBersih'].sum()

Tag_kol2 = list_pubexp[list_pubexp.Kol==5].copy()
Tag_kol2 = Tag_kol2.groupby('Ket_Sekeko')['TagihanBersih'].sum()

CkpnInd = list_pubexp.groupby('Ket_Sekeko')['CkpnIndividu'].sum()

CkpnKol = list_pubexp.groupby('Ket_Sekeko')['CkpnKol'].sum()

list_1_5 = pd.concat([Tag,Tag_kol1,Tag_kol2,CkpnInd,CkpnKol],axis=1,sort=False).reset_index()
list_1_5.reset_index(drop=True, inplace=True)
list_1_5_cols = ['Ket_Sekeko', '01.Tag', '02.Tag1', '03.Tag5', '04.CkpnInd', '05.CkpnKol']
list_1_5.columns = list_1_5_cols

# List_1_5

list_1_7 = list_pubexp[["Ket_Katport_final","Kel_Rating","TagihanBersih"]]
list_1_7 = pd.pivot_table(list_1_7,
            index=['Ket_Katport_final'],
            columns=['Kel_Rating'],
            aggfunc=[np.sum],
            values=['TagihanBersih'],
            fill_value=0,
            margins=True,
            dropna=True)
print('Sheet list_1_1 is written successfully to Excel Sheet.')
print(" ")
print('Sheet list_1_2 is written successfully to Excel Sheet.')
print(" ")
print('Sheet list_1_3 is written successfully to Excel Sheet.')
print(" ")
print('Sheet list_1_4 is written successfully to Excel Sheet.')
print(" ")
print('Sheet list_1_5 is written successfully to Excel Sheet.')
print(" ")
print('Sheet list_1_7 is written successfully to Excel Sheet.')
print(" ")


Sheet list_1_1 is written successfully to Excel Sheet.
 
Sheet list_1_2 is written successfully to Excel Sheet.
 
Sheet list_1_3 is written successfully to Excel Sheet.
 
Sheet list_1_4 is written successfully to Excel Sheet.
 
Sheet list_1_5 is written successfully to Excel Sheet.
 
Sheet list_1_7 is written successfully to Excel Sheet.
 


## **Print Sheet 1-8**

In [17]:
import pandas as pd
import numpy as np
from itertools import cycle
from datetime import date
import math
import os

pd.set_option('display.max_columns',15)

data = pd.ExcelFile("C:/Users/20150503/Downloads/PubExp_py/KPMM/@Form_ATMR.xlsx")

columns31a = ['Sisa JathTempo','RateSwap_31a','CurrencySwap_31a']
data_31a = data.parse(u'B',skiprows=623,nrows=3,usecols = 'J:L',names=columns31a)

columns31b = ['Sisa JathTempo','RateSwap_31b','CurrencySwap_31b']
data_31b = data.parse(u'B',skiprows=640,nrows=3,usecols = 'J:L',names=columns31b)

columns32 = ['Sisa JathTempo','RateSwap_32','CurrencySwap_32']
data_32 = data.parse(u'B',skiprows=662,nrows=3,usecols = 'J:L',names=columns32)

columns33 = ['Sisa JathTempo','RateSwap_33','CurrencySwap_33']
data_33 = data.parse(u'B',skiprows=683,nrows=3,usecols = 'J:L',names=columns33)

columns34a = ['Sisa JathTempo','RateSwap_34a','CurrencySwap_34a']
data_34a = data.parse(u'B',skiprows=705,nrows=3,usecols = 'J:L',names=columns34a)

columns34b = ['Sisa JathTempo','RateSwap_34b','CurrencySwap_34b']
data_34b = data.parse(u'B',skiprows=725,nrows=3,usecols = 'J:L',names=columns34b)

columns35 = ['Sisa JathTempo','RateSwap_35','CurrencySwap_35']
data_35 = data.parse(u'B',skiprows=746,nrows=3,usecols = 'J:L',names=columns35)

columns36 = ['Sisa JathTempo','RateSwap_36','CurrencySwap_36']
data_36 = data.parse(u'B',skiprows=763,nrows=3,usecols = 'J:L',names=columns36)

Rate = data_31a[['Sisa JathTempo','RateSwap_31a']].merge(
	data_31b[['Sisa JathTempo','RateSwap_31b']],on='Sisa JathTempo').merge(
	data_32[['Sisa JathTempo','RateSwap_32']],on='Sisa JathTempo').merge(
	data_33[['Sisa JathTempo','RateSwap_33']],on='Sisa JathTempo').merge(
	data_34a[['Sisa JathTempo','RateSwap_34a']],on='Sisa JathTempo').merge(
	data_34b[['Sisa JathTempo','RateSwap_34b']],on='Sisa JathTempo').merge(
	data_35[['Sisa JathTempo','RateSwap_35']],on='Sisa JathTempo').merge(
	data_36[['Sisa JathTempo','RateSwap_36']],on='Sisa JathTempo')
Rate['Shariah Compliant Profit Rate Swap'] = Rate.sum(axis=1)
# print(Rate)
Currency = data_31a[['Sisa JathTempo','CurrencySwap_31a']].merge(
	data_31b[['Sisa JathTempo','CurrencySwap_31b']],on='Sisa JathTempo').merge(
	data_32[['Sisa JathTempo','CurrencySwap_32']],on='Sisa JathTempo').merge(
	data_33[['Sisa JathTempo','CurrencySwap_33']],on='Sisa JathTempo').merge(
	data_34a[['Sisa JathTempo','CurrencySwap_34a']],on='Sisa JathTempo').merge(
	data_34b[['Sisa JathTempo','CurrencySwap_34b']],on='Sisa JathTempo').merge(
	data_35[['Sisa JathTempo','CurrencySwap_35']],on='Sisa JathTempo').merge(
	data_36[['Sisa JathTempo','CurrencySwap_36']],on='Sisa JathTempo')
Currency['Shariah Compliant Foreign Currency Swap'] = Currency.sum(axis=1)	
Rate = Rate[['Sisa JathTempo','Shariah Compliant Profit Rate Swap']]
Currency = Currency[['Sisa JathTempo','Shariah Compliant Foreign Currency Swap']]

Not_amt = pd.merge(Rate,Currency,left_on="Sisa JathTempo",right_on="Sisa JathTempo",how="left")
Not_amt.set_index('Sisa JathTempo',inplace=True)
Not_amt = Not_amt.transpose()

columns = ['Shariah Compliant Foreign Currency Swap']
tag_dervtf = data.parse(u'A',skiprows=256,nrows=1,usecols = 'H',names=columns)
tag_dervtf = tag_dervtf.rename(index={0: 'Tagihan Derivatif'})
tag_dervtf = tag_dervtf.transpose()

data_mtm = pd.ExcelFile("C:/Users/20150503/Downloads/BMPP/BMPP- 20201231-Des'20/MTM_2020-12-30.xls")
columns = ['Shariah Compliant Foreign Currency Swap']
wjb_dervtf = data_mtm.parse(u'Sheet1',skiprows=3,nrows=1,usecols = 'AC',names=columns)
wjb_dervtf = wjb_dervtf.rename(index={0: 'Kewajiban Derivatif'})
wjb_dervtf = wjb_dervtf.transpose()
wjb_dervtf = wjb_dervtf['Kewajiban Derivatif']/1_000_000


columns31a_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_31a_mrk = data.parse(u'B',skiprows=631,nrows=1,usecols = 'F,H',names=columns31a_mrk)

columns31b_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_31b_mrk = data.parse(u'B',skiprows=648,nrows=6,usecols = 'F,H',names=columns31b_mrk)

columns32_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_32_mrk = data.parse(u'B',skiprows=648,nrows=5,usecols = 'F,H',names=columns32_mrk)

columns33_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_33_mrk = data.parse(u'B',skiprows=648,nrows=6,usecols = 'F,H',names=columns33_mrk)

columns34a_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_34a_mrk = data.parse(u'B',skiprows=713,nrows=4,usecols = 'F,H',names=columns34a_mrk)

columns34b_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_34b_mrk = data.parse(u'B',skiprows=733,nrows=5,usecols = 'F,H',names=columns34b_mrk)

columns35_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_35_mrk = data.parse(u'B',skiprows=754,nrows=1,usecols = 'F,H',names=columns35_mrk)

columns36_mrk = ['Tagihan Bersih sebelum MRK','Tagihan Bersih setelah MRK']
data_36_mrk = data.parse(u'B',skiprows=771,nrows=5,usecols = 'F,H',names=columns36_mrk)

Deriftv_mrk=pd.concat([data_31a_mrk,data_31b_mrk,data_32_mrk, data_33_mrk, 
					data_34a_mrk,data_34b_mrk,data_35_mrk,data_36_mrk],axis=0,sort=False)
Deriftv_mrk.reset_index(drop=True, inplace=True)

Deriftv_mrk.loc['Shariah Compliant Foreign Currency Swap'] = Deriftv_mrk.sum()
Deriftv_mrk = Deriftv_mrk.tail(1)

list_1_8=pd.concat([Not_amt,tag_dervtf,wjb_dervtf, Deriftv_mrk],axis=1,sort=False)
list_1_8.fillna(value=0, inplace=True)
	
# print(list_1_8)
print('Sheet list_1_8 is written successfully to Excel Sheet.')






c:\program files\python38-32\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Sheet list_1_8 is written successfully to Excel Sheet.


## **Print sheet 1-9**

In [19]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',15)

columns_bobot = ['Bobot Resiko','Tagihan Bersih']
columns_tag = [0,0.2,0.5,1]
columns_atmr = ['ATMR']

data = pd.ExcelFile("C:/Users/20150503/Downloads/PubExp_py/KPMM/@Form_ATMR.xlsx")
#1-1
data_1_1a = data.parse(u'B',skiprows=14,nrows=1,usecols = 'D,H',names=columns_bobot)

data_1_1b = data.parse(u'B',skiprows=14,nrows=5,usecols = 'J:M',names=columns_tag)
data_1_1b.loc['Tagihan Bersih'] = data_1_1b.sum()
data_1_1b = data_1_1b.tail(1)
data_1_1b = data_1_1b.transpose()
data_1_1b = data_1_1b.reset_index()
data_1_1b.columns = columns_bobot

data_1_1mrk = data.parse(u'B',skiprows=17,nrows=1,usecols = 'F',names=columns_atmr)
data_1_1=pd.concat([data_1_1a,data_1_1b,data_1_1mrk],axis=0,sort=False)
data_1_1['Ket_Katport'] = '1.01 Tagihan Kepada Pemerintah Republik Indonesia'

data_1_1.fillna(value=0, inplace=True)

#1-2
data_1_2a = data.parse(u'B',skiprows=45,nrows=5,usecols = 'D,H',names=columns_bobot)

data_1_2b = data.parse(u'B',skiprows=45,nrows=5,usecols = 'J:M',names=columns_tag)
data_1_2b.loc['Tagihan Bersih'] = data_1_2b.sum()
data_1_2b = data_1_2b.tail(1)
data_1_2b = data_1_2b.transpose()
data_1_2b = data_1_2b.reset_index()
data_1_2b.columns = columns_bobot

data_1_2mrk = data.parse(u'B',skiprows=52,nrows=1,usecols = 'F',names=columns_atmr)

data_1_2=pd.concat([data_1_2a,data_1_2b,data_1_2mrk],axis=0,sort=False)
data_1_2.fillna(value=0, inplace=True)
data_1_2['Ket_Katport'] = '1.02 Tagihan Kepada Entitas Sektor Publik'

#1-3
data_1_3a = data.parse(u'B',skiprows=62,nrows=6,usecols = 'D,H',names=columns_bobot)
data_1_3b = data.parse(u'B',skiprows=62,nrows=6,usecols = 'J:M',names=columns_tag)
data_1_3b.loc['Tagihan Bersih'] = data_1_3b.sum()
data_1_3b = data_1_3b.tail(1)
data_1_3b = data_1_3b.transpose()
data_1_3b = data_1_3b.reset_index()
data_1_3b.columns = columns_bobot

data_1_3mrk = data.parse(u'B',skiprows=70,nrows=1,usecols = 'F',names=columns_atmr)

data_1_3=pd.concat([data_1_3a,data_1_3b,data_1_3mrk],axis=0,sort=False)
data_1_3.fillna(value=0, inplace=True)
data_1_3['Ket_Katport'] = '1.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'

#1-4
data_1_4a_pdk = data.parse(u'B',skiprows=80,nrows=8,usecols = 'D,H',names=columns_bobot)
data_1_4a_pjg = data.parse(u'B',skiprows=100,nrows=9,usecols = 'D,H',names=columns_bobot)

data_1_4a = pd.concat([data_1_4a_pdk, data_1_4a_pjg],axis=0,sort=False)

data_1_4b_pdk = data.parse(u'B',skiprows=80,nrows=8,usecols = 'J:M',names=columns_tag)
data_1_4b_pdk.loc['Tagihan Bersih'] = data_1_4b_pdk.sum()
data_1_4b_pdk = data_1_4b_pdk.tail(1)
data_1_4b_pdk = data_1_4b_pdk.transpose()
data_1_4b_pdk = data_1_4b_pdk.reset_index()
data_1_4b_pdk.columns = columns_bobot

data_1_4b_pjg = data.parse(u'B',skiprows=100,nrows=9,usecols = 'J:M',names=columns_tag)
data_1_4b_pjg.loc['Tagihan Bersih'] = data_1_4b_pjg.sum()
data_1_4b_pjg = data_1_4b_pjg.tail(1)
data_1_4b_pjg = data_1_4b_pjg.transpose()
data_1_4b_pjg = data_1_4b_pjg.reset_index()
data_1_4b_pjg.columns = columns_bobot

data_1_4b = pd.concat([data_1_4b_pdk, data_1_4b_pjg],axis=0,sort=False)

data_1_4mrk_pdk = data.parse(u'B',skiprows=90,nrows=1,usecols = 'F',names=columns_atmr)
data_1_4mrk_pjg = data.parse(u'B',skiprows=111,nrows=1,usecols = 'F',names=columns_atmr)

data_1_4mrk = pd.concat([data_1_4mrk_pdk, data_1_4mrk_pjg],axis=0,sort=False)

data_1_4=pd.concat([data_1_4a,data_1_4b,data_1_4mrk],axis=0,sort=False)
data_1_4.fillna(value=0, inplace=True)
data_1_4['Ket_Katport'] = '1.04 Tagihan Kepada Bank'

#1-5
data_1_5a = data.parse(u'B',skiprows=121,nrows=3,usecols = 'D,H',names=columns_bobot)

data_1_5b = data.parse(u'B',skiprows=121,nrows=3,usecols = 'J:M',names=columns_tag)
data_1_5b.loc['Tagihan Bersih'] = data_1_5b.sum()
data_1_5b = data_1_5b.tail(1)
data_1_5b = data_1_5b.transpose()
data_1_5b = data_1_5b.reset_index()
data_1_5b.columns = columns_bobot

data_1_5mrk = data.parse(u'B',skiprows=126,nrows=1,usecols = 'F',names=columns_atmr)

data_1_5=pd.concat([data_1_5a,data_1_5b,data_1_5mrk],axis=0,sort=False)
data_1_5.fillna(value=0, inplace=True)
data_1_5['Ket_Katport'] = '1.05 Pembiayaan Beragun Rumah Tinggal'

#1-6
data_1_6a = data.parse(u'B',skiprows=136,nrows=1,usecols = 'D,H',names=columns_bobot)

data_1_6b = data.parse(u'B',skiprows=136,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_6b.loc['Tagihan Bersih'] = data_1_6b.sum()
data_1_6b = data_1_6b.tail(1)
data_1_6b = data_1_6b.transpose()
data_1_6b = data_1_6b.reset_index()
data_1_6b.columns = columns_bobot

data_1_6mrk = data.parse(u'B',skiprows=139,nrows=1,usecols = 'F',names=columns_atmr)

data_1_6=pd.concat([data_1_6a,data_1_6b,data_1_6mrk],axis=0,sort=False)
data_1_6.fillna(value=0, inplace=True)
data_1_6['Ket_Katport'] = '1.06 Pembiayaan Beragun Properti Komersial'

#1-7
data_1_7a = data.parse(u'B',skiprows=149,nrows=1,usecols = 'D,H',names=columns_bobot)

data_1_7b = data.parse(u'B',skiprows=149,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_7b.loc['Tagihan Bersih'] = data_1_7b.sum()
data_1_7b = data_1_7b.tail(1)
data_1_7b = data_1_7b.transpose()
data_1_7b = data_1_7b.reset_index()
data_1_7b.columns = columns_bobot

data_1_7mrk = data.parse(u'B',skiprows=152,nrows=1,usecols = 'F',names=columns_atmr)

data_1_7=pd.concat([data_1_7a,data_1_7b,data_1_7mrk],axis=0,sort=False)
data_1_7.fillna(value=0, inplace=True)
data_1_7['Ket_Katport'] = '1.07 Pembiayaan Pegawai/Pensiunan'

#1-8
data_1_8a = data.parse(u'B',skiprows=162,nrows=1,usecols = 'D,H',names=columns_bobot)

data_1_8b = data.parse(u'B',skiprows=162,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_8b.loc['Tagihan Bersih'] = data_1_8b.sum()
data_1_8b = data_1_8b.tail(1)
data_1_8b = data_1_8b.transpose()
data_1_8b = data_1_8b.reset_index()
data_1_8b.columns = columns_bobot

data_1_8mrk = data.parse(u'B',skiprows=165,nrows=1,usecols = 'F',names=columns_atmr)

data_1_8 = pd.concat([data_1_8a,data_1_8b,data_1_8mrk],axis=0,sort=False)
data_1_8.fillna(value=0, inplace=True)
data_1_8['Ket_Katport'] = '1.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'

#1-9
data_1_9a = data.parse(u'B',skiprows=175,nrows=9,usecols = 'D,H',names=columns_bobot)

data_1_9b = data.parse(u'B',skiprows=175,nrows=9,usecols = 'J:M',names=columns_tag)
data_1_9b.loc['Tagihan Bersih'] = data_1_9b.sum()
data_1_9b = data_1_9b.tail(1)
data_1_9b = data_1_9b.transpose()
data_1_9b = data_1_9b.reset_index()
data_1_9b.columns = columns_bobot

data_1_9mrk = data.parse(u'B',skiprows=186,nrows=1,usecols = 'F',names=columns_atmr)

data_1_9 = pd.concat([data_1_9a,data_1_9b,data_1_9mrk],axis=0,sort=False)
data_1_9.fillna(value=0, inplace=True)
data_1_9['Ket_Katport'] = '1.09 Tagihan Kepada Korporasi'

#1-10
data_1_10a = data.parse(u'B',skiprows=196,nrows=2,usecols = 'D,H',names=columns_bobot)

data_1_10b = data.parse(u'B',skiprows=196,nrows=2,usecols = 'J:M',names=columns_tag)
data_1_10b.loc['Tagihan Bersih'] = data_1_10b.sum()
data_1_10b = data_1_10b.tail(1)
data_1_10b = data_1_10b.transpose()
data_1_10b = data_1_10b.reset_index()
data_1_10b.columns = columns_bobot

data_1_10mrk = data.parse(u'B',skiprows=200,nrows=1,usecols = 'F',names=columns_atmr)

data_1_10 = pd.concat([data_1_10a,data_1_10b,data_1_10mrk],axis=0,sort=False)	
data_1_10.fillna(value=0, inplace=True)
data_1_10['Ket_Katport'] = '1.10 Tagihan Yang Telah Jatuh Tempo'

data_1=pd.concat([data_1_1,data_1_2,data_1_3,data_1_4,data_1_5,data_1_6,data_1_7,data_1_8,data_1_9,data_1_10],axis=0,sort=False)
list_1_9a= pd.pivot_table(data_1,
            index=['Ket_Katport'],
            columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['Tagihan Bersih'],
            fill_value=0,
            # margins=True,
            dropna=True)
list_1_9a_mrk= pd.pivot_table(data_1,
            index=['Ket_Katport'],
            # columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['ATMR'],
            fill_value=0,
            # margins=True,
            dropna=True)

# data_1=pd.concat([data_11,data_1mrk],axis=1,sort=False)
# print(list_1_9a)
# print(list_1_9a_mrk)

#2-1
data_2_1a = data.parse(u'B',skiprows=260,nrows=1,usecols = 'D,H',names=columns_bobot)

data_2_1b = data.parse(u'B',skiprows=260,nrows=1,usecols = 'J:M',names=columns_tag)
data_2_1b.loc['Tagihan Bersih'] = data_2_1b.sum()
data_2_1b = data_2_1b.tail(1)
data_2_1b = data_2_1b.transpose()
data_2_1b = data_2_1b.reset_index()
data_2_1b.columns = columns_bobot

data_2_1mrk = data.parse(u'B',skiprows=263,nrows=1,usecols = 'F',names=columns_atmr)

data_2_1 = pd.concat([data_2_1a,data_2_1b,data_2_1mrk],axis=0,sort=False)	
data_2_1.fillna(value=0, inplace=True)
data_2_1['Ket_Katport'] = '2.01 Tagihan Kepada Pemerintah Republik Indonesia'

#2-2
data_2_2a = data.parse(u'B',skiprows=315,nrows=5,usecols = 'D,H',names=columns_bobot)

data_2_2b = data.parse(u'B',skiprows=315,nrows=5,usecols = 'J:M',names=columns_tag)
data_2_2b.loc['Tagihan Bersih'] = data_2_2b.sum()
data_2_2b = data_2_2b.tail(1)
data_2_2b = data_2_2b.transpose()
data_2_2b = data_2_2b.reset_index()
data_2_2b.columns = columns_bobot

data_2_2mrk = data.parse(u'B',skiprows=322,nrows=1,usecols = 'F',names=columns_atmr)

data_2_2 = pd.concat([data_2_2a,data_2_2b,data_2_2mrk],axis=0,sort=False)	
data_2_2.fillna(value=0, inplace=True)
data_2_2['Ket_Katport'] = '2.02 Tagihan Kepada Entitas Sektor Publik'

#2-3
data_2_3a = data.parse(u'B',skiprows=344,nrows=6,usecols = 'D,H',names=columns_bobot)

data_2_3b = data.parse(u'B',skiprows=344,nrows=6,usecols = 'J:M',names=columns_tag)
data_2_3b.loc['Tagihan Bersih'] = data_2_3b.sum()
data_2_3b = data_2_3b.tail(1)
data_2_3b = data_2_3b.transpose()
data_2_3b = data_2_3b.reset_index()
data_2_3b.columns = columns_bobot

data_2_3mrk = data.parse(u'B',skiprows=352,nrows=1,usecols = 'F',names=columns_atmr)

data_2_3 = pd.concat([data_2_3a,data_2_3b,data_2_3mrk],axis=0,sort=False)	
data_2_3.fillna(value=0, inplace=True)
data_2_3['Ket_Katport'] = '2.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'

#2-4
data_2_4a_pdk = data.parse(u'B',skiprows=374,nrows=8,usecols = 'D,H',names=columns_bobot)
data_2_4a_pjg = data.parse(u'B',skiprows=406,nrows=9,usecols = 'D,H',names=columns_bobot)

data_2_4a = pd.concat([data_2_4a_pdk, data_2_4a_pjg],axis=0,sort=False)

data_2_4b_pdk = data.parse(u'B',skiprows=374,nrows=8,usecols = 'J:M',names=columns_tag)
data_2_4b_pdk.loc['Tagihan Bersih'] = data_2_4b_pdk.sum()
data_2_4b_pdk = data_2_4b_pdk.tail(1)
data_2_4b_pdk = data_2_4b_pdk.transpose()
data_2_4b_pdk = data_2_4b_pdk.reset_index()
data_2_4b_pdk.columns = columns_bobot

data_2_4b_pjg = data.parse(u'B',skiprows=406,nrows=9,usecols = 'J:M',names=columns_tag)
data_2_4b_pjg.loc['Tagihan Bersih'] = data_2_4b_pjg.sum()
data_2_4b_pjg = data_2_4b_pjg.tail(1)
data_2_4b_pjg = data_2_4b_pjg.transpose()
data_2_4b_pjg = data_2_4b_pjg.reset_index()
data_2_4b_pjg.columns = columns_bobot

data_2_4b = pd.concat([data_2_4b_pdk, data_2_4b_pjg],axis=0,sort=False)

data_2_4mrk_pdk = data.parse(u'B',skiprows=384,nrows=1,usecols = 'F',names=columns_atmr)
data_2_4mrk_pjg = data.parse(u'B',skiprows=417,nrows=1,usecols = 'F',names=columns_atmr)

data_2_4mrk = pd.concat([data_2_4mrk_pdk, data_2_4mrk_pjg],axis=0,sort=False)

data_2_4=pd.concat([data_2_4a,data_2_4b,data_2_4mrk],axis=0,sort=False)
data_2_4.fillna(value=0, inplace=True)
data_2_4['Ket_Katport'] = '2.04 Tagihan Kepada Bank'

#2-5
data_2_5a = data.parse(u'B',skiprows=435,nrows=3,usecols = 'D,H',names=columns_bobot)

data_2_5b = data.parse(u'B',skiprows=435,nrows=3,usecols = 'J:M',names=columns_tag)
data_2_5b.loc['Tagihan Bersih'] = data_2_5b.sum()
data_2_5b = data_2_5b.tail(1)
data_2_5b = data_2_5b.transpose()
data_2_5b = data_2_5b.reset_index()
data_2_5b.columns = columns_bobot

data_2_5mrk = data.parse(u'B',skiprows=440,nrows=1,usecols = 'F',names=columns_atmr)

data_2_5=pd.concat([data_2_5a,data_2_5b,data_2_5mrk],axis=0,sort=False)
data_2_5.fillna(value=0, inplace=True)
data_2_5['Ket_Katport'] = '2.05 Pembiayaan Beragun Rumah Tinggal'

#2-6
data_2_6a = data.parse(u'B',skiprows=458,nrows=1,usecols = 'D,H',names=columns_bobot)

data_2_6b = data.parse(u'B',skiprows=458,nrows=1,usecols = 'J:M',names=columns_tag)
data_2_6b.loc['Tagihan Bersih'] = data_2_6b.sum()
data_2_6b = data_2_6b.tail(1)
data_2_6b = data_2_6b.transpose()
data_2_6b = data_2_6b.reset_index()
data_2_6b.columns = columns_bobot

data_2_6mrk = data.parse(u'B',skiprows=461,nrows=1,usecols = 'F',names=columns_atmr)

data_2_6=pd.concat([data_2_6a,data_2_6b,data_2_6mrk],axis=0,sort=False)
data_2_6.fillna(value=0, inplace=True)
data_2_6['Ket_Katport'] = '2.06 Pembiayaan Beragun Properti Komersial'

#2-7
data_2_7a = data.parse(u'B',skiprows=479,nrows=1,usecols = 'D,H',names=columns_bobot)

data_2_7b = data.parse(u'B',skiprows=479,nrows=1,usecols = 'J:M',names=columns_tag)
data_2_7b.loc['Tagihan Bersih'] = data_2_7b.sum()
data_2_7b = data_2_7b.tail(1)
data_2_7b = data_2_7b.transpose()
data_2_7b = data_2_7b.reset_index()
data_2_7b.columns = columns_bobot

data_2_7mrk = data.parse(u'B',skiprows=482,nrows=1,usecols = 'F',names=columns_atmr)

data_2_7=pd.concat([data_2_7a,data_2_7b,data_2_7mrk],axis=0,sort=False)
data_2_7.fillna(value=0, inplace=True)
data_2_7['Ket_Katport'] = '2.07 Pembiayaan Pegawai/Pensiunan'

#2-8
data_2_8a = data.parse(u'B',skiprows=504,nrows=1,usecols = 'D,H',names=columns_bobot)

data_2_8b = data.parse(u'B',skiprows=504,nrows=1,usecols = 'J:M',names=columns_tag)
data_2_8b.loc['Tagihan Bersih'] = data_2_8b.sum()
data_2_8b = data_2_8b.tail(1)
data_2_8b = data_2_8b.transpose()
data_2_8b = data_2_8b.reset_index()
data_2_8b.columns = columns_bobot

data_2_8mrk = data.parse(u'B',skiprows=507,nrows=1,usecols = 'F',names=columns_atmr)

data_2_8 = pd.concat([data_2_8a,data_2_8b,data_2_8mrk],axis=0,sort=False)
data_2_8.fillna(value=0, inplace=True)
data_2_8['Ket_Katport'] = '2.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'

#2-9
data_2_9a = data.parse(u'B',skiprows=529,nrows=9,usecols = 'D,H',names=columns_bobot)

data_2_9b = data.parse(u'B',skiprows=529,nrows=9,usecols = 'J:M',names=columns_tag)
data_2_9b.loc['Tagihan Bersih'] = data_2_9b.sum()
data_2_9b = data_2_9b.tail(1)
data_2_9b = data_2_9b.transpose()
data_2_9b = data_2_9b.reset_index()
data_2_9b.columns = columns_bobot

data_2_9mrk = data.parse(u'B',skiprows=540,nrows=1,usecols = 'F',names=columns_atmr)

data_2_9 = pd.concat([data_2_9a,data_2_9b,data_2_9mrk],axis=0,sort=False)
data_2_9.fillna(value=0, inplace=True)
data_2_9['Ket_Katport'] = '2.09 Tagihan Kepada Korporasi'

#1-10
data_2_10a = data.parse(u'B',skiprows=559,nrows=2,usecols = 'D,H',names=columns_bobot)

data_2_10b = data.parse(u'B',skiprows=559,nrows=2,usecols = 'J:M',names=columns_tag)
data_2_10b.loc['Tagihan Bersih'] = data_2_10b.sum()
data_2_10b = data_2_10b.tail(1)
data_2_10b = data_2_10b.transpose()
data_2_10b = data_2_10b.reset_index()
data_2_10b.columns = columns_bobot

data_2_10mrk = data.parse(u'B',skiprows=563,nrows=1,usecols = 'F',names=columns_atmr)

data_2_10 = pd.concat([data_2_10a,data_2_10b,data_2_10mrk],axis=0,sort=False)	
data_2_10.fillna(value=0, inplace=True)
data_2_10['Ket_Katport'] = '2.10 Tagihan Yang Telah Jatuh Tempo'


data_2=pd.concat([data_2_1,data_2_2,data_2_3,data_2_4,data_2_5,data_2_6,data_2_7,data_2_8,data_2_9,data_2_10],axis=0,sort=False)
list_1_9b= pd.pivot_table(data_2,
            index=['Ket_Katport'],
            columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['Tagihan Bersih'],
            fill_value=0,
            # margins=True,
            dropna=True)
list_1_9b_mrk= pd.pivot_table(data_2,
            index=['Ket_Katport'],
            # columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['ATMR'],
            fill_value=0,
            # margins=True,
            dropna=True)

# data_1=pd.concat([data_11,data_1mrk],axis=1,sort=False)
# print(list_1_9b)
# print(list_1_9b_mrk)

#3-1
data_3_1a = data.parse(u'B',skiprows=631,nrows=1,usecols = 'D,H',names=columns_bobot)
data_3_1mrk = data.parse(u'B',skiprows=634,nrows=1,usecols = 'F',names=columns_atmr)

data_3_1 = pd.concat([data_3_1a,data_3_1mrk],axis=0,sort=False)	
data_3_1.fillna(value=0, inplace=True)
data_3_1['Ket_Katport'] = '3.01 Tagihan Kepada Pemerintah Republik Indonesia'

#3-2
data_3_2a = data.parse(u'B',skiprows=670,nrows=5,usecols = 'D,H',names=columns_bobot)
data_3_2mrk = data.parse(u'B',skiprows=677,nrows=1,usecols = 'F',names=columns_atmr)

data_3_2 = pd.concat([data_3_2a,data_3_2mrk],axis=0,sort=False)	
data_3_2.fillna(value=0, inplace=True)
data_3_2['Ket_Katport'] = '3.02 Tagihan Kepada Entitas Sektor Publik'

#3-3
data_3_3a = data.parse(u'B',skiprows=670,nrows=5,usecols = 'D,H',names=columns_bobot)
data_3_3mrk = data.parse(u'B',skiprows=677,nrows=1,usecols = 'F',names=columns_atmr)

data_3_3 = pd.concat([data_3_3a,data_3_3mrk],axis=0,sort=False)	
data_3_3.fillna(value=0, inplace=True)
data_3_3['Ket_Katport'] = '3.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'

#3-4
data_3_4a_pdk = data.parse(u'B',skiprows=713,nrows=4,usecols = 'D,H',names=columns_bobot)
data_3_4a_pjg = data.parse(u'B',skiprows=733,nrows=5,usecols = 'D,H',names=columns_bobot)

data_3_4a = pd.concat([data_3_4a_pdk, data_3_4a_pjg],axis=0,sort=False)

data_3_4mrk_pdk = data.parse(u'B',skiprows=719,nrows=1,usecols = 'F',names=columns_atmr)
data_3_4mrk_pjg = data.parse(u'B',skiprows=740,nrows=1,usecols = 'F',names=columns_atmr)

data_3_4mrk = pd.concat([data_3_4mrk_pdk, data_3_4mrk_pjg],axis=0,sort=False)

data_3_4=pd.concat([data_3_4a,data_3_4mrk],axis=0,sort=False)
data_3_4.fillna(value=0, inplace=True)
data_3_4['Ket_Katport'] = '3.04 Tagihan Kepada Bank'

#3-5
data_3_5a = data.parse(u'B',skiprows=754,nrows=1,usecols = 'D,H',names=columns_bobot)
data_3_5mrk = data.parse(u'B',skiprows=757,nrows=1,usecols = 'F',names=columns_atmr)

data_3_5=pd.concat([data_3_5a,data_3_5mrk],axis=0,sort=False)
data_3_5.fillna(value=0, inplace=True)
data_3_5['Ket_Katport'] = '3.05 Pembiayaan Beragun Rumah Tinggal'

#3-6
data_3_6a = data.parse(u'B',skiprows=771,nrows=5,usecols = 'D,H',names=columns_bobot)
data_3_6mrk = data.parse(u'B',skiprows=778,nrows=1,usecols = 'F',names=columns_atmr)

data_3_6=pd.concat([data_3_6a,data_3_6mrk],axis=0,sort=False)
data_3_6.fillna(value=0, inplace=True)
data_3_6['Ket_Katport'] = '3.06 Pembiayaan Beragun Properti Komersial'

data_3=pd.concat([data_3_1,data_3_2,data_3_3,data_3_4,data_3_5,data_3_6],axis=0,sort=False)
list_1_9c= pd.pivot_table(data_3,
            index=['Ket_Katport'],
            columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['Tagihan Bersih'],
            fill_value=0,
            # margins=True,
            dropna=True)
list_1_9c_mrk= pd.pivot_table(data_3,
            index=['Ket_Katport'],
            # columns=['Bobot Resiko'],
            aggfunc=[sum],
            values=['ATMR'],
            fill_value=0,
            # margins=True,
            dropna=True)

# data_1=pd.concat([data_11,data_1mrk],axis=1,sort=False)
# print(list_1_9c)
# print(list_1_9c_mrk)

print('Sheet list_1_9 is written successfully to Excel Sheet.')


c:\program files\python38-32\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\program files\python38-32\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\program files\python38-32\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Sheet list_1_9 is written successfully to Excel Sheet.


## **Print Sheet 1-10**

In [20]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',15)

columns_bobot = ['Tagihan Bersih']
columns_tag = [0,0.2,0.5,1]
# columns_atmr = ['ATMR']

data = pd.ExcelFile("C:/Users/20150503/Downloads/PubExp_py/KPMM/@Form_ATMR.xlsx")

#1-1
data_1_1a = data.parse(u'B',skiprows=9,nrows=1,usecols = 'D',names=columns_bobot)
data_1_1b = data.parse(u'B',skiprows=14,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_1b['Agunan'] = data_1_1b.sum()
data_1_1b = data_1_1b.head(1)
data_1_1b = data_1_1b[["Agunan"]]
data_1_1=pd.concat([data_1_1a,data_1_1b],axis=1,sort=False)
data_1_1['Ket_Katport'] = '1.01 Tagihan Kepada Pemerintah Republik Indonesia'
#1-2
data_1_2a = data.parse(u'B',skiprows=40,nrows=1,usecols = 'D',names=columns_bobot)
data_1_2b = data.parse(u'B',skiprows=45,nrows=5,usecols = 'J:M',names=columns_tag)
data_1_2b['Agunan'] = data_1_2b.sum()
data_1_2b = data_1_2b.head(1)
data_1_2b = data_1_2b[["Agunan"]]
data_1_2=pd.concat([data_1_2a,data_1_2b],axis=1,sort=False)
data_1_2['Ket_Katport'] = '1.02 Tagihan Kepada Entitas Sektor Publik'
#1-3
data_1_3a = data.parse(u'B',skiprows=57,nrows=1,usecols = 'D',names=columns_bobot)
data_1_3b = data.parse(u'B',skiprows=62,nrows=6,usecols = 'J:M',names=columns_tag)
data_1_3b['Agunan'] = data_1_3b.sum()
data_1_3b = data_1_3b.head(1)
data_1_3b = data_1_3b[["Agunan"]]
data_1_3=pd.concat([data_1_3a,data_1_3b],axis=1,sort=False)
data_1_3['Ket_Katport'] = '1.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'
#1-4
data_1_4a_pdk = data.parse(u'B',skiprows=75,nrows=1,usecols = 'D',names=columns_bobot)
data_1_4a_pjg = data.parse(u'B',skiprows=95,nrows=1,usecols = 'D',names=columns_bobot)
data_1_4a = pd.concat([data_1_4a_pdk, data_1_4a_pjg],axis=0,sort=False)
data_1_4a.loc['Tagihan Bersih'] = data_1_4a.sum()
data_1_4a = data_1_4a.tail(1)
data_1_4a = data_1_4a.transpose()
data_1_4a = data_1_4a.reset_index()
data_1_4a = data_1_4a[["Tagihan Bersih"]]

data_1_4b_pdk = data.parse(u'B',skiprows=80,nrows=8,usecols = 'J:M',names=columns_tag)
data_1_4b_pjg = data.parse(u'B',skiprows=100,nrows=9,usecols = 'J:M',names=columns_tag)
data_1_4b = pd.concat([data_1_4b_pdk, data_1_4b_pjg],axis=0,sort=False)

data_1_4b['Agunan'] = data_1_4b.sum()
data_1_4b = data_1_4b.head(1)
data_1_4b = data_1_4b[["Agunan"]]
data_1_4=pd.concat([data_1_4a,data_1_4b],axis=1,sort=False)
data_1_4['Ket_Katport'] = '1.04 Tagihan Kepada Bank'
#1-5
data_1_5a = data.parse(u'B',skiprows=116,nrows=1,usecols = 'D',names=columns_bobot)
data_1_5b = data.parse(u'B',skiprows=121,nrows=3,usecols = 'J:M',names=columns_tag)
data_1_5b.loc['Agunan'] = data_1_5b.sum(axis=0)
data_1_5b = data_1_5b.tail(1)
data_1_5b = data_1_5b.transpose()
data_1_5b.loc['Agunan'] = data_1_5b.sum()
data_1_5b = data_1_5b.tail(1)
data_1_5b = data_1_5b.transpose()
data_1_5b = data_1_5b.reset_index()
data_1_5b = data_1_5b[["Agunan"]]

data_1_5=pd.concat([data_1_5a,data_1_5b],axis=1,sort=False)
data_1_5['Ket_Katport'] = '1.05 Pembiayaan Beragun Rumah Tinggal'
data_1=pd.concat([data_1_1,data_1_2,data_1_3,data_1_4,data_1_5],axis=0,sort=False)
data_1 = data_1[["Ket_Katport","Tagihan Bersih","Agunan"]]
#1-6
data_1_6a = data.parse(u'B',skiprows=131,nrows=1,usecols = 'D',names=columns_bobot)
data_1_6b = data.parse(u'B',skiprows=136,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_6b['Agunan'] = data_1_6b.sum()
data_1_6b = data_1_6b.head(1)
data_1_6b = data_1_6b[["Agunan"]]
data_1_6=pd.concat([data_1_6a,data_1_6b],axis=1,sort=False)
data_1_6['Ket_Katport'] = '1.06 Pembiayaan Beragun Properti Komersial'
#1-7
data_1_7a = data.parse(u'B',skiprows=144,nrows=1,usecols = 'D',names=columns_bobot)
data_1_7b = data.parse(u'B',skiprows=149,nrows=1,usecols = 'J:M',names=columns_tag)
data_1_7b['Agunan'] = data_1_7b.sum()
data_1_7b = data_1_7b.head(1)
data_1_7b = data_1_7b[["Agunan"]]
data_1_7=pd.concat([data_1_7a,data_1_7b],axis=1,sort=False)
data_1_7['Ket_Katport'] = '1.07 Pembiayaan Pegawai/Pensiunan'
#1-8
data_1_8a = data.parse(u'B',skiprows=157,nrows=1,usecols = 'D',names=columns_bobot)
data_1_8b = data.parse(u'B',skiprows=162,nrows=1,usecols = 'J:M',names=columns_tag)

data_1_8b.loc['Agunan'] = data_1_8b.sum(axis=0)
data_1_8b = data_1_8b.tail(1)
data_1_8b = data_1_8b.transpose()
data_1_8b.loc['Agunan'] = data_1_8b.sum()
data_1_8b = data_1_8b.tail(1)
data_1_8b = data_1_8b.transpose()
data_1_8b = data_1_8b.reset_index()
data_1_8b = data_1_8b[["Agunan"]]
data_1_8=pd.concat([data_1_8a,data_1_8b],axis=1,sort=False)
data_1_8['Ket_Katport'] = '1.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'
#1-9
data_1_9a = data.parse(u'B',skiprows=170,nrows=1,usecols = 'D',names=columns_bobot)
data_1_9b = data.parse(u'B',skiprows=175,nrows=9,usecols = 'J:M',names=columns_tag)
data_1_9b.loc['Agunan'] = data_1_9b.sum(axis=0)
data_1_9b = data_1_9b.tail(1)
data_1_9b = data_1_9b.transpose()
data_1_9b.loc['Agunan'] = data_1_9b.sum()
data_1_9b = data_1_9b.tail(1)
data_1_9b = data_1_9b.transpose()
data_1_9b = data_1_9b.reset_index()
data_1_9b = data_1_9b[["Agunan"]]
data_1_9=pd.concat([data_1_9a,data_1_9b],axis=1,sort=False)
data_1_9['Ket_Katport'] = '1.09 Tagihan Kepada Korporasi'
#1-10
data_1_10a = data.parse(u'B',skiprows=191,nrows=1,usecols = 'D',names=columns_bobot)
data_1_10b = data.parse(u'B',skiprows=196,nrows=9,usecols = 'J:M',names=columns_tag)
data_1_10b['Agunan'] = data_1_10b.sum()
data_1_10b = data_1_10b.head(1)
data_1_10b = data_1_10b[["Agunan"]]
data_1_10=pd.concat([data_1_10a,data_1_10b],axis=1,sort=False)
data_1_10['Ket_Katport'] = '1.10 Tagihan Yang Telah Jatuh Tempo'

data_1=pd.concat([data_1_1,data_1_2,data_1_3,data_1_4,data_1_5,data_1_6,data_1_7,data_1_8,data_1_9,data_1_10],axis=0,sort=False)
data_1=data_1[["Ket_Katport","Tagihan Bersih","Agunan"]]
data_1.loc['Total Part A'] = data_1.sum(numeric_only=True, axis=0)
data_1.fillna(value='', inplace=True)

# print(data_1)

#2-1
data_2_1 = data.parse(u'B',skiprows=255,nrows=1,usecols = 'H',names=columns_bobot)
data_2_1['Ket_Katport'] = '2.01 Tagihan Kepada Pemerintah Indonesia'
#2-2
data_2_2 = data.parse(u'B',skiprows=310,nrows=1,usecols = 'H',names=columns_bobot)
data_2_2['Ket_Katport'] = '2.02 Tagihan Kepada Entitas Sektor Publik'
#2-3
data_2_3 = data.parse(u'B',skiprows=339,nrows=1,usecols = 'H',names=columns_bobot)
data_2_3['Ket_Katport'] = '2.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'
#2-4
data_2_4_pdk = data.parse(u'B',skiprows=369,nrows=1,usecols = 'H',names=columns_bobot)
data_2_4_pjg = data.parse(u'B',skiprows=401,nrows=1,usecols = 'H',names=columns_bobot)
data_2_4 = pd.concat([data_2_4_pdk, data_2_4_pjg],axis=0,sort=False)

data_2_4.loc['Tagihan Bersih'] = data_2_4.sum(axis=0)
data_2_4 = data_2_4.tail(1)
data_2_4 = data_2_4.reset_index()
data_2_4 = data_2_4[["Tagihan Bersih"]]
data_2_4['Ket_Katport'] = '2.04 Tagihan Kepada Bank'
#2-5
data_2_5 = data.parse(u'B',skiprows=430,nrows=1,usecols = 'H',names=columns_bobot)
data_2_5['Ket_Katport'] = '2.05 Pembiayaan Beragun Rumah Tinggal'
#2-6
data_2_6 = data.parse(u'B',skiprows=453,nrows=1,usecols = 'H',names=columns_bobot)
data_2_6['Ket_Katport'] = '2.06 Pembiayaan Beragun Properti Komersial'
#2-7
data_2_7 = data.parse(u'B',skiprows=474,nrows=1,usecols = 'H',names=columns_bobot)
data_2_7['Ket_Katport'] = '2.07 Pembiayaan Pegawai/Pensiunan'
#2-8
data_2_8 = data.parse(u'B',skiprows=499,nrows=1,usecols = 'H',names=columns_bobot)
data_2_8['Ket_Katport'] = '2.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'
#2-9
data_2_9 = data.parse(u'B',skiprows=524,nrows=1,usecols = 'H',names=columns_bobot)
data_2_9['Ket_Katport'] = '2.09 Tagihan Kepada Korporasi'
#2-10
data_2_10 = data.parse(u'B',skiprows=554,nrows=1,usecols = 'H',names=columns_bobot)
data_2_10['Ket_Katport'] = '2.10 Tagihan Yang Telah Jatuh Tempo'

data_2=pd.concat([data_2_1,data_2_2,data_2_3,data_2_4,data_2_5,data_2_6,data_2_7,data_2_8,data_2_9,data_2_10],axis=0,sort=False)
data_2=data_2[["Ket_Katport","Tagihan Bersih"]]
data_2.loc['Total Part B'] = data_2.sum(numeric_only=True, axis=0)
data_2.fillna(value='', inplace=True)

# print(data_2)

#3-1
data_3_1 = data.parse(u'B',skiprows=626,nrows=1,usecols = 'H',names=columns_bobot)
data_3_1['Ket_Katport'] = '3.01 Tagihan Kepada Pemerintah Indonesia'
#3-2
data_3_2 = data.parse(u'B',skiprows=665,nrows=1,usecols = 'H',names=columns_bobot)
data_3_2['Ket_Katport'] = '3.02 Tagihan Kepada Entitas Sektor Publik'
#3-3
data_3_3 = data.parse(u'B',skiprows=686,nrows=1,usecols = 'H',names=columns_bobot)
data_3_3['Ket_Katport'] = '3.03 Tagihan Kepada Bank Pembangunan Multilateral dan Lembaga Internasional'
#3-4
data_3_4_pdk = data.parse(u'B',skiprows=707,nrows=1,usecols = 'H',names=columns_bobot)
data_3_4_pjg = data.parse(u'B',skiprows=728,nrows=1,usecols = 'H',names=columns_bobot)
data_3_4 = pd.concat([data_3_4_pdk, data_3_4_pjg],axis=0,sort=False)

data_3_4.loc['Tagihan Bersih'] = data_3_4.sum(axis=0)
data_3_4 = data_3_4.tail(1)
data_3_4 = data_3_4.reset_index()
data_3_4 = data_3_4[["Tagihan Bersih"]]
data_3_4['Ket_Katport'] = '3.04 Tagihan Kepada Bank'
#3-5
data_3_5 = data.parse(u'B',skiprows=749,nrows=1,usecols = 'H',names=columns_bobot)
data_3_5['Ket_Katport'] = '3.05 Pembiayaan Beragun Rumah Tinggal'
#3-6
data_3_6 = data.parse(u'B',skiprows=766,nrows=1,usecols = 'H',names=columns_bobot)
data_3_6['Ket_Katport'] = '3.06 Pembiayaan Beragun Properti Komersial'

data_3=pd.concat([data_3_1,data_3_2,data_3_3,data_3_4,data_3_5,data_3_6],axis=0,sort=False)
data_3=data_3[["Ket_Katport","Tagihan Bersih"]]
data_3.loc['Total Part C'] = data_3.sum(numeric_only=True, axis=0)
data_3.fillna(value='', inplace=True)

# print(data_3)

list_1_10 = pd.concat([data_1,data_2,data_3],axis=0,sort=False)
list_1_10.fillna(value='', inplace=True)

# print(list_1_10)
print('Sheet list_1_10 is written successfully to Excel Sheet.')



c:\program files\python38-32\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Sheet list_1_10 is written successfully to Excel Sheet.


## **Print Sheet 1-13**

In [21]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',15)

columns_bobot = ['Tagihan Bersih']
columns_tag = [0,0.2,0.5,1]
# columns_atmr = ['ATMR']

data = pd.ExcelFile("C:/Users/20150503/Downloads/PubExp_py/KPMM/@Form_ATMR.xlsx")


#1-1
data_1_1a = data.parse(u'B',skiprows=9,nrows=1,usecols = 'D',names=columns_bobot)

data_1_1b = data.parse(u'B',skiprows=16,nrows=2,usecols = 'F',names=columns_bobot)
data_1_1b = data_1_1b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_1b = data_1_1b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_1b = data_1_1b.transpose()
data_1_1b = data_1_1b.reset_index()
data_1_1b.drop('index', axis=1, inplace=True)
data_1_1=pd.concat([data_1_1a,data_1_1b],axis=1,sort=False)
data_1_1['Ket_Katport'] = '1.01 Tagihan Kepada Pemerintah Republik Indonesia'

#1-2
data_1_2a = data.parse(u'B',skiprows=40,nrows=1,usecols = 'D',names=columns_bobot)

data_1_2b = data.parse(u'B',skiprows=51,nrows=2,usecols = 'F',names=columns_bobot)
data_1_2b = data_1_2b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_2b = data_1_2b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_2b = data_1_2b.transpose()
data_1_2b = data_1_2b.reset_index()
data_1_2b.drop('index', axis=1, inplace=True)
data_1_2=pd.concat([data_1_2a,data_1_2b],axis=1,sort=False)
data_1_2['Ket_Katport'] = '1.02 Tagihan Kepada Entitas Sektor Publik'

#1-3
data_1_3a = data.parse(u'B',skiprows=57,nrows=1,usecols = 'D',names=columns_bobot)

data_1_3b = data.parse(u'B',skiprows=69,nrows=2,usecols = 'F',names=columns_bobot)
data_1_3b = data_1_3b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_3b = data_1_3b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_3b = data_1_3b.transpose()
data_1_3b = data_1_3b.reset_index()
data_1_3b.drop('index', axis=1, inplace=True)
data_1_3=pd.concat([data_1_3a,data_1_3b],axis=1,sort=False)
data_1_3['Ket_Katport'] = '1.03 Tagihan Kepada Pemerintah Republik Indonesia'

#1-4
data_1_4a_pdk = data.parse(u'B',skiprows=75,nrows=1,usecols = 'D',names=columns_bobot)
data_1_4a_pjg = data.parse(u'B',skiprows=95,nrows=1,usecols = 'D',names=columns_bobot)
data_1_4a = pd.concat([data_1_4a_pdk,data_1_4a_pjg],axis=0,sort=False)
data_1_4a.loc['Tagihan Bersih'] = data_1_4a.sum()
data_1_4a = data_1_4a.tail(1)
data_1_4a = data_1_4a.transpose()
data_1_4a = data_1_4a.reset_index()
data_1_4a = data_1_4a[["Tagihan Bersih"]]

data_1_4b_pdk = data.parse(u'B',skiprows=89,nrows=2,usecols = 'F',names=columns_bobot)
data_1_4b_pjg = data.parse(u'B',skiprows=110,nrows=2,usecols = 'F',names=columns_bobot)
data_1_4b = pd.concat([data_1_4b_pdk,data_1_4b_pjg],axis=1,sort=False)
data_1_4b['Tagihan'] = data_1_4b.sum(axis=1)
data_1_4b = data_1_4b[['Tagihan']]
data_1_4b = data_1_4b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_4b = data_1_4b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_4b = data_1_4b.transpose()
data_1_4b = data_1_4b.reset_index()
data_1_4b.drop('index', axis=1, inplace=True)
data_1_4=pd.concat([data_1_4a,data_1_4b],axis=1,sort=False)
data_1_4['Ket_Katport'] = '1.04 Tagihan Kepada Bank'

#1-5
data_1_5a = data.parse(u'B',skiprows=116,nrows=1,usecols = 'D',names=columns_bobot)

data_1_5b = data.parse(u'B',skiprows=125,nrows=2,usecols = 'F',names=columns_bobot)
data_1_5b = data_1_5b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_5b = data_1_5b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_5b = data_1_5b.transpose()
data_1_5b = data_1_5b.reset_index()
data_1_5b.drop('index', axis=1, inplace=True)
data_1_5=pd.concat([data_1_5a,data_1_5b],axis=1,sort=False)
data_1_5['Ket_Katport'] = '1.05 Pembiayaan Beragun Rumah Tinggal'

#1-6
data_1_6a = data.parse(u'B',skiprows=131,nrows=1,usecols = 'D',names=columns_bobot)

data_1_6b = data.parse(u'B',skiprows=138,nrows=2,usecols = 'F',names=columns_bobot)
data_1_6b = data_1_6b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_6b = data_1_6b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_6b = data_1_6b.transpose()
data_1_6b = data_1_6b.reset_index()
data_1_6b.drop('index', axis=1, inplace=True)
data_1_6=pd.concat([data_1_6a,data_1_6b],axis=1,sort=False)
data_1_6['Ket_Katport'] = '1.06 Pembiayaan Beragun Properti Komersial'

#1-7
data_1_7a = data.parse(u'B',skiprows=144,nrows=1,usecols = 'D',names=columns_bobot)

data_1_7b = data.parse(u'B',skiprows=151,nrows=2,usecols = 'F',names=columns_bobot)
data_1_7b = data_1_7b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_7b = data_1_7b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_7b = data_1_7b.transpose()
data_1_7b = data_1_7b.reset_index()
data_1_7b.drop('index', axis=1, inplace=True)
data_1_7=pd.concat([data_1_7a,data_1_7b],axis=1,sort=False)
data_1_7['Ket_Katport'] = '1.07 Pembiayaan Pegawai/Pensiunan'

#1-8
data_1_8a = data.parse(u'B',skiprows=157,nrows=1,usecols = 'D',names=columns_bobot)

data_1_8b = data.parse(u'B',skiprows=164,nrows=2,usecols = 'F',names=columns_bobot)
data_1_8b = data_1_8b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_8b = data_1_8b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_8b = data_1_8b.transpose()
data_1_8b = data_1_8b.reset_index()
data_1_8b.drop('index', axis=1, inplace=True)
data_1_8=pd.concat([data_1_8a,data_1_8b],axis=1,sort=False)
data_1_8['Ket_Katport'] = '1.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'

#1-9
data_1_9a = data.parse(u'B',skiprows=170,nrows=1,usecols = 'D',names=columns_bobot)

data_1_9b = data.parse(u'B',skiprows=185,nrows=2,usecols = 'F',names=columns_bobot)
data_1_9b = data_1_9b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_9b = data_1_9b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_9b = data_1_9b.transpose()
data_1_9b = data_1_9b.reset_index()
data_1_9b.drop('index', axis=1, inplace=True)
data_1_9=pd.concat([data_1_9a,data_1_9b],axis=1,sort=False)
data_1_9['Ket_Katport'] = '1.09 Tagihan Kepada Korporasi'

#1-10
data_1_10a = data.parse(u'B',skiprows=191,nrows=1,usecols = 'D',names=columns_bobot)

data_1_10b = data.parse(u'B',skiprows=199,nrows=2,usecols = 'F',names=columns_bobot)
data_1_10b = data_1_10b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_1_10b = data_1_10b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_1_10b = data_1_10b.transpose()
data_1_10b = data_1_10b.reset_index()
data_1_10b.drop('index', axis=1, inplace=True)
data_1_10=pd.concat([data_1_10a,data_1_10b],axis=1,sort=False)
data_1_10['Ket_Katport'] = '1.10 Tagihan Yang Telah Jatuh Tempo'

data_1=pd.concat([data_1_1,data_1_2,data_1_3,data_1_4,data_1_5,data_1_6,data_1_7,data_1_8,data_1_9,data_1_10],axis=0,sort=False)
data_1=data_1[["Ket_Katport","Tagihan Bersih","Total ATMR sebelum pengakuan MRK","Total ATMR setelah pengakuan MRK"]]
data_1.loc['Total Part A'] = data_1.sum(numeric_only=True, axis=0)
data_1.fillna(value='', inplace=True)

# print(data_1)


#2-1
data_2_1a = data.parse(u'B',skiprows=255,nrows=1,usecols = 'H',names=columns_bobot)

data_2_1b = data.parse(u'B',skiprows=262,nrows=2,usecols = 'F',names=columns_bobot)
data_2_1b = data_2_1b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_1b = data_2_1b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_1b = data_2_1b.transpose()
data_2_1b = data_2_1b.reset_index()
data_2_1b.drop('index', axis=1, inplace=True)
data_2_1=pd.concat([data_2_1a,data_2_1b],axis=1,sort=False)
data_2_1['Ket_Katport'] = '2.01 Tagihan Kepada Pemerintah Republik Indonesia'
 
#2-2
data_2_2a = data.parse(u'B',skiprows=310,nrows=1,usecols = 'H',names=columns_bobot)

data_2_2b = data.parse(u'B',skiprows=321,nrows=2,usecols = 'F',names=columns_bobot)
data_2_2b = data_2_2b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_2b = data_2_2b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_2b = data_2_2b.transpose()
data_2_2b = data_2_2b.reset_index()
data_2_2b.drop('index', axis=1, inplace=True)
data_2_2=pd.concat([data_2_2a,data_2_2b],axis=1,sort=False)
data_2_2['Ket_Katport'] = '2.02 Tagihan Kepada Entitas Sektor Publik'
 
#2-3
data_2_3a = data.parse(u'B',skiprows=339,nrows=1,usecols = 'H',names=columns_bobot)

data_2_3b = data.parse(u'B',skiprows=351,nrows=2,usecols = 'F',names=columns_bobot)
data_2_3b = data_2_3b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_3b = data_2_3b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_3b = data_2_3b.transpose()
data_2_3b = data_2_3b.reset_index()
data_2_3b.drop('index', axis=1, inplace=True)
data_2_3=pd.concat([data_2_3a,data_2_3b],axis=1,sort=False)
data_2_3['Ket_Katport'] = '2.03 Tagihan Kepada Pemerintah Republik Indonesia'
 
#1-4
data_2_4a_pdk = data.parse(u'B',skiprows=369,nrows=1,usecols = 'H',names=columns_bobot)
data_2_4a_pjg = data.parse(u'B',skiprows=401,nrows=1,usecols = 'H',names=columns_bobot)
data_2_4a = pd.concat([data_2_4a_pdk,data_2_4a_pjg],axis=0,sort=False)
data_2_4a.loc['Tagihan Bersih'] = data_2_4a.sum()
data_2_4a = data_2_4a.tail(1)
data_2_4a = data_2_4a.transpose()
data_2_4a = data_2_4a.reset_index()
data_2_4a = data_2_4a[["Tagihan Bersih"]]

data_2_4b_pdk = data.parse(u'B',skiprows=384,nrows=2,usecols = 'F',names=columns_bobot)
data_2_4b_pjg = data.parse(u'B',skiprows=417,nrows=2,usecols = 'F',names=columns_bobot)
data_2_4b = pd.concat([data_2_4b_pdk,data_2_4b_pjg],axis=1,sort=False)
data_2_4b['Tagihan'] = data_2_4b.sum(axis=1)
data_2_4b = data_2_4b[['Tagihan']]
data_2_4b = data_2_4b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_4b = data_2_4b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_4b = data_2_4b.transpose()
data_2_4b = data_2_4b.reset_index()
data_2_4b.drop('index', axis=1, inplace=True)
data_2_4=pd.concat([data_2_4a,data_2_4b],axis=1,sort=False)
data_2_4['Ket_Katport'] = '2.04 Tagihan Kepada Bank'
 
#2-5
data_2_5a = data.parse(u'B',skiprows=430,nrows=1,usecols = 'H',names=columns_bobot)

data_2_5b = data.parse(u'B',skiprows=439,nrows=2,usecols = 'F',names=columns_bobot)
data_2_5b = data_2_5b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_5b = data_2_5b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_5b = data_2_5b.transpose()
data_2_5b = data_2_5b.reset_index()
data_2_5b.drop('index', axis=1, inplace=True)
data_2_5=pd.concat([data_2_5a,data_2_5b],axis=1,sort=False)
data_2_5['Ket_Katport'] = '2.05 Pembiayaan Beragun Rumah Tinggal'
 
#2-6
data_2_6a = data.parse(u'B',skiprows=453,nrows=1,usecols = 'H',names=columns_bobot)

data_2_6b = data.parse(u'B',skiprows=460,nrows=2,usecols = 'F',names=columns_bobot)
data_2_6b = data_2_6b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_6b = data_2_6b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_6b = data_2_6b.transpose()
data_2_6b = data_2_6b.reset_index()
data_2_6b.drop('index', axis=1, inplace=True)
data_2_6=pd.concat([data_2_6a,data_2_6b],axis=1,sort=False)
data_2_6['Ket_Katport'] = '2.06 Pembiayaan Beragun Properti Komersial'
 
#2-7
data_2_7a = data.parse(u'B',skiprows=474,nrows=1,usecols = 'H',names=columns_bobot)

data_2_7b = data.parse(u'B',skiprows=481,nrows=2,usecols = 'F',names=columns_bobot)
data_2_7b = data_2_7b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_7b = data_2_7b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_7b = data_2_7b.transpose()
data_2_7b = data_2_7b.reset_index()
data_2_7b.drop('index', axis=1, inplace=True)
data_2_7=pd.concat([data_2_7a,data_2_7b],axis=1,sort=False)
data_2_7['Ket_Katport'] = '2.07 Pembiayaan Pegawai/Pensiunan'
 
#2-8
data_2_8a = data.parse(u'B',skiprows=499,nrows=1,usecols = 'H',names=columns_bobot)

data_2_8b = data.parse(u'B',skiprows=506,nrows=2,usecols = 'F',names=columns_bobot)
data_2_8b = data_2_8b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_8b = data_2_8b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_8b = data_2_8b.transpose()
data_2_8b = data_2_8b.reset_index()
data_2_8b.drop('index', axis=1, inplace=True)
data_2_8=pd.concat([data_2_8a,data_2_8b],axis=1,sort=False)
data_2_8['Ket_Katport'] = '2.08 Tagihan Kepada Usaha Mikro, Usaha Kecil dan Portofolio Ritel'
 
#2-9
data_2_9a = data.parse(u'B',skiprows=524,nrows=1,usecols = 'H',names=columns_bobot)

data_2_9b = data.parse(u'B',skiprows=539,nrows=2,usecols = 'F',names=columns_bobot)
data_2_9b = data_2_9b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_9b = data_2_9b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_9b = data_2_9b.transpose()
data_2_9b = data_2_9b.reset_index()
data_2_9b.drop('index', axis=1, inplace=True)
data_2_9=pd.concat([data_2_9a,data_2_9b],axis=1,sort=False)
data_2_9['Ket_Katport'] = '2.09 Tagihan Kepada Korporasi'
 
#2-10
data_2_10a = data.parse(u'B',skiprows=554,nrows=1,usecols = 'H',names=columns_bobot)

data_2_10b = data.parse(u'B',skiprows=562,nrows=2,usecols = 'F',names=columns_bobot)
data_2_10b = data_2_10b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_2_10b = data_2_10b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_2_10b = data_2_10b.transpose()
data_2_10b = data_2_10b.reset_index()
data_2_10b.drop('index', axis=1, inplace=True)
data_2_10=pd.concat([data_2_10a,data_2_10b],axis=1,sort=False)
data_2_10['Ket_Katport'] = '2.10 Tagihan Yang Telah Jatuh Tempo'
 
data_2 = pd.concat([data_2_1,data_2_2,data_2_3,data_2_4,data_2_5,data_2_6,data_2_7,data_2_8,data_2_9,data_2_10],axis=0,sort=False)
data_2 = data_2[["Ket_Katport","Tagihan Bersih","Total ATMR sebelum pengakuan MRK","Total ATMR setelah pengakuan MRK"]]
data_2.loc['Total Part B'] = data_2.sum(numeric_only=True, axis=0)
data_2.fillna(value='', inplace=True)
# print(data_2)

#3-1
data_3_1a = data.parse(u'B',skiprows=626,nrows=1,usecols = 'H',names=columns_bobot)

data_3_1b = data.parse(u'B',skiprows=633,nrows=2,usecols = 'F',names=columns_bobot)
data_3_1b = data_3_1b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_1b = data_3_1b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_1b = data_3_1b.transpose()
data_3_1b = data_3_1b.reset_index()
data_3_1b.drop('index', axis=1, inplace=True)
data_3_1=pd.concat([data_3_1a,data_3_1b],axis=1,sort=False)
data_3_1['Ket_Katport'] = '3.01 Tagihan Kepada Pemerintah Republik Indonesia'
 
#3-2
data_3_2a = data.parse(u'B',skiprows=665,nrows=1,usecols = 'H',names=columns_bobot)

data_3_2b = data.parse(u'B',skiprows=676,nrows=2,usecols = 'F',names=columns_bobot)
data_3_2b = data_3_2b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_2b = data_3_2b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_2b = data_3_2b.transpose()
data_3_2b = data_3_2b.reset_index()
data_3_2b.drop('index', axis=1, inplace=True)
data_3_2=pd.concat([data_3_2a,data_3_2b],axis=1,sort=False)
data_3_2['Ket_Katport'] = '3.02 Tagihan Kepada Entitas Sektor Publik'
 
#3-3
data_3_3a = data.parse(u'B',skiprows=686,nrows=1,usecols = 'H',names=columns_bobot)

data_3_3b = data.parse(u'B',skiprows=698,nrows=2,usecols = 'F',names=columns_bobot)
data_3_3b = data_3_3b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_3b = data_3_3b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_3b = data_3_3b.transpose()
data_3_3b = data_3_3b.reset_index()
data_3_3b.drop('index', axis=1, inplace=True)
data_3_3=pd.concat([data_3_3a,data_3_3b],axis=1,sort=False)
data_3_3['Ket_Katport'] = '3.03 Tagihan Kepada Pemerintah Republik Indonesia'
 
#3-4
data_3_4a_pdk = data.parse(u'B',skiprows=708,nrows=1,usecols = 'H',names=columns_bobot)
data_3_4a_pjg = data.parse(u'B',skiprows=728,nrows=1,usecols = 'H',names=columns_bobot)
data_3_4a = pd.concat([data_3_4a_pdk,data_3_4a_pjg],axis=0,sort=False)
data_3_4a.loc['Tagihan Bersih'] = data_3_4a.sum()
data_3_4a = data_3_4a.tail(1)
data_3_4a = data_3_4a.transpose()
data_3_4a = data_3_4a.reset_index()
data_3_4a = data_3_4a[["Tagihan Bersih"]]

data_3_4b_pdk = data.parse(u'B',skiprows=718,nrows=2,usecols = 'F',names=columns_bobot)
data_3_4b_pjg = data.parse(u'B',skiprows=739,nrows=2,usecols = 'F',names=columns_bobot)
data_3_4b = pd.concat([data_3_4b_pdk,data_3_4b_pjg],axis=1,sort=False)
data_3_4b['Tagihan'] = data_3_4b.sum(axis=1)
data_3_4b = data_3_4b[['Tagihan']]
data_3_4b = data_3_4b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_4b = data_3_4b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_4b = data_3_4b.transpose()
data_3_4b = data_3_4b.reset_index()
data_3_4b.drop('index', axis=1, inplace=True)
data_3_4=pd.concat([data_3_4a,data_3_4b],axis=1,sort=False)
data_3_4['Ket_Katport'] = '3.04 Tagihan Kepada Bank'
 
#3-5
data_3_5a = data.parse(u'B',skiprows=749,nrows=1,usecols = 'H',names=columns_bobot)

data_3_5b = data.parse(u'B',skiprows=756,nrows=2,usecols = 'F',names=columns_bobot)
data_3_5b = data_2_5b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_5b = data_2_5b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_5b = data_2_5b.transpose()
data_3_5b = data_2_5b.reset_index()
data_3_5b.drop('index', axis=1, inplace=True)
data_3_5=pd.concat([data_3_5a,data_3_5b],axis=1,sort=False)
data_3_5['Ket_Katport'] = '3.05 Pembiayaan Beragun Rumah Tinggal'
 
#3-6
data_3_6a = data.parse(u'B',skiprows=766,nrows=1,usecols = 'H',names=columns_bobot)

data_3_6b = data.parse(u'B',skiprows=777,nrows=2,usecols = 'F',names=columns_bobot)
data_3_6b = data_3_6b.rename(index={0:'Total ATMR sebelum pengakuan MRK'})
data_3_6b = data_3_6b.rename(index={1:'Total ATMR setelah pengakuan MRK'})
data_3_6b = data_3_6b.transpose()
data_3_6b = data_3_6b.reset_index()
data_3_6b.drop('index', axis=1, inplace=True)
data_3_6=pd.concat([data_3_6a,data_3_6b],axis=1,sort=False)
data_3_6['Ket_Katport'] = '3.06 Pembiayaan Beragun Properti Komersial'

data_3 = pd.concat([data_3_1,data_3_2,data_3_3,data_3_4,data_3_5,data_3_6],axis=0,sort=False)
data_3 = data_3[["Ket_Katport","Tagihan Bersih","Total ATMR sebelum pengakuan MRK","Total ATMR setelah pengakuan MRK"]]
data_3.loc['Total Part C'] = data_3.sum(numeric_only=True, axis=0)
data_3.fillna(value='', inplace=True)
# print(data_3)

list_1_13 = pd.concat([data_1,data_2,data_3],axis=0,sort=False)
# print(list_1_13)
print('Sheet list_1_13 is written successfully to Excel Sheet.')


Sheet list_1_13 is written successfully to Excel Sheet.


## **Print Sheet Resik Pasar Standar**

In [22]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',15)

col = ['tagihan']
data = pd.ExcelFile("C:/Users/20150503/Downloads/PubExp_py/KPMM/@Form_ATMR.xlsx")

resiko_umum = data.parse(u'V.A_Solo',skiprows=5,nrows=1,usecols = 'C',names=col)
resiko_umum = resiko_umum.rename(index={0:'b. Risiko Umum'})
resiko_tukar = data.parse(u'V.A_Solo',skiprows=5,nrows=1,usecols = 'D',names=col)
resiko_tukar = resiko_tukar.rename(index={0:'Risiko Nilai Tukar'})

resiko_pasar_standar = pd.concat([resiko_umum, resiko_tukar],axis=0,sort=False)
print('Sheet 2_RESIKO_PASAR_STANDAR is written successfully to Excel Sheet.')


Sheet 2_RESIKO_PASAR_STANDAR is written successfully to Excel Sheet.


In [23]:
# create excel writer
with pd.ExcelWriter('C:/Users/20150503/Downloads/PubExp_py/OUTPUT/brpp_pubex_template1.xlsx') as dt:
    KPMM_sheet_A.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'KPMM_sheet_A')
    KPMM_sheet_A1.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'KPMM_sheet_A1')

    BRBB.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'Summ_brbb')
    list_1_1.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_1')
    list_1_2.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_2')
    list_1_3.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_3')
    list_1_4.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_4')
    list_1_5.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_5')
    list_1_7.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_7')
    list_1_8.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_8')
    list_1_9a.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_9')
    list_1_9a_mrk.to_excel   (dt, startrow=0, startcol=10, sheet_name = 'list_1_9')
    list_1_9b.to_excel   (dt, startrow=15, startcol=0, sheet_name = 'list_1_9')
    list_1_9b_mrk.to_excel   (dt, startrow=15, startcol=10, sheet_name = 'list_1_9')
    list_1_9c.to_excel   (dt, startrow=30, startcol=0, sheet_name = 'list_1_9')
    list_1_9c_mrk.to_excel   (dt, startrow=30, startcol=10, sheet_name = 'list_1_9')
    list_1_10.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_10')   
    list_1_13.to_excel   (dt, startrow=0, startcol=0, sheet_name = 'list_1_13')   
    resiko_pasar_standar.to_excel   (dt, startrow=0, startcol=0, sheet_name = '2_RESIKO_PASAR_STANDAR')
    dt.save()
print("Data Publikasi Exposure has done export to Excell template ...")


Data Publikasi Exposure has done export to Excell template ...
